In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from datasets import Dataset
import time

import transformers
# from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertTokenizerFast
from transformers import BertTokenizerFast, BertForSequenceClassification

import torch
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
from torch.optim import AdamW
from torch.utils.data.dataset import ConcatDataset

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


from imblearn.over_sampling import SMOTE

# Data Analytics

In [ ]:

concatenated_df = pd.read_csv('chunked_new.csv')

del concatenated_df['Unnamed: 0']
concatenated_df

,LogContent,RootCause
0,metrics2. impl. metricsconfig : loaded propert...,MachineDown
1,. maptask : ( equator ) 0 kvi 26214396 ( 10485...,MachineDown
2,##k : kvstart = 11165712 ( 44662848 ) ; kvend ...,MachineDown
3,: kvstart = 7281300 ( 29125200 ) ; kvend = 210...,MachineDown
4,##388 ( 13585552 ) ; kvend = 17183192 ( 687327...,MachineDown
...,...,...
4369,maptask : ( equator ) 0 kvi 26214396 ( 1048575...,MachineDown
4370,kvstart = 14330776 ( 57323104 ) ; kvend = 5417...,MachineDown
4371,##t = 16762552 ( 67050208 ) ; kvend = 7853256 ...,MachineDown
4372,##4 ( 76802656 ) ; kvend = 10290256 ( 41161024...,MachineDown


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
concatenated_df['tokens'] = concatenated_df['LogContent'].apply(lambda x : tokenizer.encode(x))

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
concatenated_df['token_numbers'] = concatenated_df['tokens'].apply(lambda x : len(x))

In [ ]:
concatenated_df

,LogContent,RootCause,tokens,token_numbers
0,metrics2. impl. metricsconfig : loaded propert...,MachineDown,"[101, 12046, 2015, 2475, 1012, 17727, 2140, 10...",510
1,. maptask : ( equator ) 0 kvi 26214396 ( 10485...,MachineDown,"[101, 1012, 4949, 10230, 2243, 1024, 1006, 266...",510
2,##k : kvstart = 11165712 ( 44662848 ) ; kvend ...,MachineDown,"[101, 1001, 1001, 1047, 1024, 24888, 14117, 21...",512
3,: kvstart = 7281300 ( 29125200 ) ; kvend = 210...,MachineDown,"[101, 1024, 24888, 14117, 2102, 1027, 5824, 26...",510
4,##388 ( 13585552 ) ; kvend = 17183192 ( 687327...,MachineDown,"[101, 1001, 1001, 4229, 2620, 1006, 11502, 275...",421
...,...,...,...,...
4369,maptask : ( equator ) 0 kvi 26214396 ( 1048575...,MachineDown,"[101, 4949, 10230, 2243, 1024, 1006, 26640, 10...",510
4370,kvstart = 14330776 ( 57323104 ) ; kvend = 5417...,MachineDown,"[101, 24888, 14117, 2102, 1027, 16065, 14142, ...",510
4371,##t = 16762552 ( 67050208 ) ; kvend = 7853256 ...,MachineDown,"[101, 1001, 1001, 1056, 1027, 16785, 2575, 177...",512
4372,##4 ( 76802656 ) ; kvend = 10290256 ( 41161024...,MachineDown,"[101, 1001, 1001, 1018, 1006, 6146, 17914, 238...",512


In [ ]:
concatenated_df.to_csv('forAnalysis_cunked.csv')

# Start The Pre Processing Phase 1

In [ ]:
import re

In [ ]:
Filter_time = data
# Filter_time['LogContent'] = Filter_time['LogContent'].str.replace(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3} ', '')
Filter_time['LogContent'] = Filter_time['LogContent'].apply(lambda x: re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}', '', x))
# Display the DataFrame

In [ ]:
# Filter_time['LogContent'] = Filter_time['LogContent'].apply(lambda x: re.sub(r'INFO', '', x))
# Filter_time['LogContent'] = Filter_time['LogContent'].apply(lambda x: re.sub(r'org.apache.hadoop.', '', x))
Filter_time['LogContent'] = Filter_time['LogContent'].apply(lambda x: re.sub(r'\[main\]', '', x))

In [ ]:
data['LogContent'][0]

5140

In [ ]:
Filter_time['LogContent'][0]

"   metrics2.impl.MetricsConfig: loaded properties from hadoop-metrics2.properties\n   metrics2.impl.MetricsSystemImpl: Scheduled snapshot period at 10 second(s).\n   metrics2.impl.MetricsSystemImpl: MapTask metrics system started\n   mapred.YarnChild: Executing with tokens:\n   mapred.YarnChild: Kind: mapreduce.job, Service: job_1445144423722_0022, Ident: (mapreduce.security.token.JobTokenIdentifier@6ace4625)\n   mapred.YarnChild: Sleeping for 0ms before retrying again. Got null now.\n   mapred.YarnChild: mapreduce.cluster.local.dir for child: /tmp/hadoop-msrabi/nm-local-dir/usercache/msrabi/appcache/application_1445144423722_0022\n   conf.Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id\n   yarn.util.ProcfsBasedProcessTree: ProcfsBasedProcessTree currently is supported only on Linux.\n   mapred.Task:  Using ResourceCalculatorProcessTree : yarn.util.WindowsBasedProcessTree@3c367995\n   mapred.MapTask: Processing split: hdfs://msra-sa-41:9000/pag

In [ ]:
Filter_time.to_csv("check.csv")

# Pre Processing Phase 2

In [ ]:
#1st time LangChain
data_process = pd.read_csv("check.csv")
del data_process['Unnamed: 0.1']
del data_process['Unnamed: 0']
del data_process['FileName']
data_process

,LogContent,RootCause
0,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection
1,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection
2,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection
3,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection
4,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection
...,...,...
753,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
754,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
755,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
756,metrics2.impl.MetricsConfig: loaded propert...,MachineDown


In [ ]:
#2nd time LangChain
df_1 = pd.read_csv('dataset.csv')
df_2 = pd.read_csv('largerLogs.csv')
data_process = pd.concat([df_1, df_2], ignore_index=True)
del data_process['Unnamed: 0']
del data_process['num_words']
del data_process['labels']
data_process

,LogContent,RootCause
0,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
1,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection
2,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
3,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection
4,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
...,...,...
621,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
622,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
623,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
624,metrics2.impl.MetricsConfig: loaded propert...,MachineDown


In [ ]:
data_process['num_words'] = data_process['LogContent'].apply(lambda x: len(x.split()))
data_process

,LogContent,RootCause,num_words
0,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,270
1,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,464
2,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,449
3,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,254
4,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,382
...,...,...,...
753,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,256
754,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,298
755,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,464
756,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,214


In [ ]:
tmp = data_process[data_process['num_words']>512]
# tmp.to_csv('largerLogs.csv')
tmp

,LogContent,RootCause,num_words
10,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1212
11,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,960
16,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,979
28,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,945
29,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,945
...,...,...,...
710,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,1074
718,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,720
728,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,997
741,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,832


In [ ]:
#test
data_process['LogContent'][0]

"   metrics2.impl.MetricsConfig: loaded properties from hadoop-metrics2.properties\n   metrics2.impl.MetricsSystemImpl: Scheduled snapshot period at 10 second(s).\n   metrics2.impl.MetricsSystemImpl: MapTask metrics system started\n   mapred.YarnChild: Executing with tokens:\n   mapred.YarnChild: Kind: mapreduce.job, Service: job_1445144423722_0022, Ident: (mapreduce.security.token.JobTokenIdentifier@6ace4625)\n   mapred.YarnChild: Sleeping for 0ms before retrying again. Got null now.\n   mapred.YarnChild: mapreduce.cluster.local.dir for child: /tmp/hadoop-msrabi/nm-local-dir/usercache/msrabi/appcache/application_1445144423722_0022\n   conf.Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id\n   yarn.util.ProcfsBasedProcessTree: ProcfsBasedProcessTree currently is supported only on Linux.\n   mapred.Task:  Using ResourceCalculatorProcessTree : yarn.util.WindowsBasedProcessTree@3c367995\n   mapred.MapTask: Processing split: hdfs://msra-sa-41:9000/pag

In [ ]:
# tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased', add_prefix_space=True)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=512)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertModel.from_pretrained("bert-base-uncased")

In [ ]:
encoded_input = tokenizer(text, return_tensors='tf', add_special_tokens=False)
encoded_input.input_ids.shape[1]

15

In [ ]:
text = "I want to know the number of tokens in this sentence!!!"
tokenized_input = tokenizer.tokenize(text)
len(tokenized_input)
tokenized_input

['i',
 'want',
 'to',
 'know',
 'the',
 'number',
 'of',
 'token',
 '##s',
 'in',
 'this',
 'sentence',
 '!',
 '!',
 '!']

In [ ]:
encoding = tokenizer.encode(text)
tokenizer.convert_ids_to_tokens(encoding)

['[CLS]',
 'i',
 'want',
 'to',
 'know',
 'the',
 'number',
 'of',
 'token',
 '##s',
 'in',
 'this',
 'sentence',
 '!',
 '!',
 '!',
 '[SEP]']

In [ ]:
data_process['tokens'] = data_process['LogContent'].apply(lambda x: tokenizer.convert_ids_to_tokens(tokenizer.encode(x)))

Token indices sequence length is longer than the specified maximum sequence length for this model (1114 > 512). Running this sequence through the model will result in indexing errors


### LangChain

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
# text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=50, chunk_overlap=10)

In [ ]:
from langchain.text_splitter import SpacyTextSplitter

text_splitter = SpacyTextSplitter(chunk_size=1200)

In [ ]:
Text = data_process['LogContent'][0]
texts = text_splitter.split_text(Text)
# print(len(str(texts[3])))
# print(texts[1])
# print(texts[2])
# print(texts[8])
type(texts)

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


list

In [ ]:
for item in texts:
  t = tokenizer.convert_ids_to_tokens(tokenizer.encode(item))
  print(len(t))

NameError: ignored

In [ ]:
data_process['TextChunks'] = data_process['LogContent'].apply(lambda x: text_splitter.split_text(x))

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
data_process.head(5)

,LogContent,RootCause,TextChunks
0,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,[metrics2.impl.\n\nMetricsConfig: loaded prope...
1,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,[metrics2.impl.\n\nMetricsConfig: loaded prope...
2,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,[metrics2.impl.\n\nMetricsConfig: loaded prope...
3,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,[metrics2.impl.\n\nMetricsConfig: loaded prope...
4,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,[metrics2.impl.\n\nMetricsConfig: loaded prope...


In [ ]:
chunked_logs = pd.DataFrame(columns=['LogContent', 'RootCause'])
tmp = []
for index, row in data_process.iterrows():
    root_cause = row['RootCause']
    log_chunks = row['TextChunks']

    # Create a DataFrame for the current row
    df = pd.DataFrame({'LogContent': log_chunks, 'RootCause': [root_cause] * len(log_chunks)})

    # Append to the list
    tmp.append(df)

# Concatenate the list of DataFrames into a single DataFrame
chunked_logs = pd.concat(tmp, ignore_index=True)

# Reorder the columns
chunked_logs = chunked_logs[['LogContent', 'RootCause']]

In [ ]:
# Apply tokenizer.convert_ids_to_tokens and calculate the length of tokens
chunked_logs['token_numbers'] = chunked_logs['LogContent'].apply(lambda x: len(tokenizer.convert_ids_to_tokens(tokenizer.encode(x))))

In [ ]:
chunked_logs[chunked_logs['token_numbers']>512]
# chunked_logs

,LogContent,RootCause,token_numbers
3165,JobHistoryEventHandler:\n\nStopping JobHistory...,MachineDown,513


In [ ]:
final_df = chunked_logs[chunked_logs['token_numbers']<=512]
# final_df.to_csv('forClassify_2.csv')
final_df.to_csv('chunked_dataset.csv')

# **Splitter**

In [ ]:
df = pd.read_csv('forClassify_1.csv')
df_2 = pd.read_csv('largerLogs.csv')
del df['Unnamed: 0.2']
del df['Unnamed: 0.1']
del df['Unnamed: 0']
del df['tokens']
del df['FileName']
del df_2['Unnamed: 0']
del df_2['num_words']
concatenated_df = pd.concat([df, df_2], ignore_index=True)
concatenated_df
labels = concatenated_df['RootCause'].unique().tolist()
labels = [s.strip() for s in labels ]
NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}
concatenated_df["labels"]=concatenated_df.RootCause.map(lambda x: label2id[x.strip()])
NUM_LABELS
concatenated_df

,LogContent,RootCause,labels
0,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,0
1,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,0
2,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,0
3,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,0
4,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,0
...,...,...,...
753,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,2
754,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,2
755,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,2
756,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,2


In [ ]:
# Split the data into draft and evaluation datasets (80-20 split)
draft, evaluation = train_test_split(concatenated_df, test_size=0.2, random_state=45, shuffle=True)
draft = draft.reset_index(drop=True)
evaluation = evaluation.reset_index(drop=True)
draft['labels'].value_counts()

2    396
1    108
0    102
Name: labels, dtype: int64

In [ ]:
evaluation['labels'].value_counts()

2    94
1    33
0    25
Name: labels, dtype: int64

In [ ]:
del evaluation['labels']
del draft['labels']

In [ ]:
evaluation.to_csv('evaluation.csv')
draft.to_csv('dataset.csv')

# FROM JUPYTER

In [ ]:
df = pd.read_csv('forClassify_2.csv')
# df = pd.read_csv('forClassify_1.csv')
# df.columns
# del df['Unnamed: 0.2']
# del df['Unnamed: 0.1']
del df['Unnamed: 0']
# del df['tokens']
# del df['FileName']
del df['token_numbers']
df

,LogContent,RootCause
0,metrics2.impl.\n\nMetricsConfig: loaded proper...,NetworkDisconnection
1,MapTask: (EQUATOR) 0\n\nkvi 26214396(104857584...,NetworkDisconnection
2,MapTask: (EQUATOR) 9726918\n\nkvi 2431724(9726...,NetworkDisconnection
3,metrics2.impl.\n\nMetricsConfig: loaded proper...,NetworkDisconnection
4,MapTask: (EQUATOR) 0\n\nkvi 26214396(104857584...,NetworkDisconnection
...,...,...
4858,[Thread-78] mapreduce.v2.app.MRAppMaster: We a...,MachineDown
4859,CompletedMaps:10 CompletedReds:1\n\nContAlloc:...,MachineDown
4860,JobHistoryEventHandler:\n\nCopied to done loca...,MachineDown
4861,JobHistoryEventHandler:\n\nMoved tmp to done: ...,MachineDown


In [ ]:
df = df[df['token_numbers'] >= 400]
del df['token_numbers']
df = df.reset_index(drop=True)
df

KeyError: ignored

In [ ]:
labels = df['RootCause'].unique().tolist()
labels = [s.strip() for s in labels ]
NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}
df["labels"]=df.RootCause.map(lambda x: label2id[x.strip()])
NUM_LABELS
label2id

{'NetworkDisconnection': 0, 'DiskFull': 1, 'MachineDown': 2}

In [ ]:
# tokenizer = BertTokenizerFast.from_pretrained("bert-large-uncased", max_length=512)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=512)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


# class CustomDataset(Dataset):
#     def __init__(self, texts, labels, tokenizer, max_length=512):
#         self.encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: val[idx] for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])  # Convert label to tensor if not already
#         return item

#     def __len__(self):
#         return len(self.labels)



### **RAW datasplit**

In [ ]:
# Tokenize the LogContent and convert labels to IDs using CustomDataset
# train_dataset = CustomDataset(texts=list(df['LogContent']), labels=[label2id[label] for label in df['RootCause']], tokenizer=tokenizer)


# Tokenize the LogContent and convert labels to IDs using CustomDataset
train_dataset = CustomDataset(texts=list(df['LogContent']), labels=df['labels'], tokenizer=tokenizer)

In [ ]:
# Split the dataset into train and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

In [ ]:
# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)
len(train_dataloader)

487

In [ ]:
# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=NUM_LABELS
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()
device

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device(type='cuda')

### Training Loop

In [ ]:
import time

# Training loop
num_epochs = 4
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    start_time = time.time()

    # Iterate over batches in the training DataLoader
    for batch_idx, batch in enumerate(train_dataloader):
        batch_start_time = time.time()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # Print training progress for each batch
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

    average_loss = total_loss / len(train_dataloader)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - start_time

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")

    # Validation
    model.eval()
    val_loss = 0.0
    correct_preds = 0

    # Iterate over batches in the validation DataLoader
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Calculate accuracy
            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += (preds == labels).sum().item()

    average_val_loss = val_loss / len(val_dataloader)
    accuracy = correct_preds / len(val_dataset)
    print(f"Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}")


<ipython-input-6-db7b66662265>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/4, Batch 1/66, Loss: 1.0857, Time: 0.84 seconds
Epoch 1/4, Batch 2/66, Loss: 0.6184, Time: 0.81 seconds
Epoch 1/4, Batch 3/66, Loss: 0.4269, Time: 0.82 seconds
Epoch 1/4, Batch 4/66, Loss: 1.0671, Time: 0.81 seconds
Epoch 1/4, Batch 5/66, Loss: 1.5483, Time: 0.81 seconds
Epoch 1/4, Batch 6/66, Loss: 1.1265, Time: 0.81 seconds
Epoch 1/4, Batch 7/66, Loss: 0.7454, Time: 0.83 seconds
Epoch 1/4, Batch 8/66, Loss: 0.4829, Time: 0.83 seconds
Epoch 1/4, Batch 9/66, Loss: 0.8772, Time: 0.83 seconds
Epoch 1/4, Batch 10/66, Loss: 1.3706, Time: 0.83 seconds
Epoch 1/4, Batch 11/66, Loss: 0.5985, Time: 0.83 seconds
Epoch 1/4, Batch 12/66, Loss: 1.0739, Time: 0.83 seconds
Epoch 1/4, Batch 13/66, Loss: 1.3496, Time: 0.84 seconds
Epoch 1/4, Batch 14/66, Loss: 1.1021, Time: 0.84 seconds
Epoch 1/4, Batch 15/66, Loss: 0.7665, Time: 0.84 seconds
Epoch 1/4, Batch 16/66, Loss: 0.8412, Time: 0.84 seconds
Epoch 1/4, Batch 17/66, Loss: 0.5336, Time: 0.85 seconds
Epoch 1/4, Batch 18/66, Loss: 0.9461, Ti

In [ ]:
# model.save_pretrained("./trained_models/old_way")

### Using class weights

In [ ]:
# Initialize the BERT model for sequence classification
M = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=NUM_LABELS
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
M.to(device)

# Define optimizer and loss function
optimizer = AdamW(M.parameters(), lr=5e-5)
# loss_fn = torch.nn.CrossEntropyLoss()
device

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device(type='cuda')

In [ ]:
class_counts = [0, 0, 0]

# Iterate over the training dataset to count samples for each class
for sample in train_dataset:
    label = sample['labels']  # Replace 'label' with the actual key used in your dataset
    class_counts[label] += 1

# Print the counts for each class
for class_idx, count in enumerate(class_counts):
    print(f"Class {class_idx}: {count} samples")


Class 0: 73 samples
Class 1: 110 samples
Class 2: 344 samples


<ipython-input-18-db7b66662265>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
# Compute class weights
class_weights = torch.tensor([1.0 / 73.0, 1.0 / 110.0, 1.0 / 344.0], dtype=torch.float).to(device)

# Define the loss function with class weights
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
# Training loop
num_epochs = 4
for epoch in range(num_epochs):
    M.train()
    total_loss = 0.0
    start_time = time.time()

    # Iterate over batches in the training DataLoader
    for batch_idx, batch in enumerate(train_dataloader):
        batch_start_time = time.time()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = M(input_ids, attention_mask=attention_mask, labels=labels)
        loss = loss_fn(outputs.logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # Print training progress for each batch
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

    average_loss = total_loss / len(train_dataloader)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - start_time

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")
    # Validation
    M.eval()
    val_loss = 0.0
    correct_preds = 0

    # Iterate over batches in the validation DataLoader
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = M(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += loss_fn(outputs.logits, labels).item()

            # Calculate accuracy
            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += (preds == labels).sum().item()

    average_val_loss = val_loss / len(val_dataloader)
    accuracy = correct_preds / len(val_dataset)
    print(f"Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}")


<ipython-input-18-db7b66662265>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/4, Batch 1/66, Loss: 0.8959, Time: 0.86 seconds
Epoch 1/4, Batch 2/66, Loss: 1.2486, Time: 0.82 seconds
Epoch 1/4, Batch 3/66, Loss: 1.1195, Time: 0.82 seconds
Epoch 1/4, Batch 4/66, Loss: 1.2306, Time: 0.82 seconds
Epoch 1/4, Batch 5/66, Loss: 1.1822, Time: 0.82 seconds
Epoch 1/4, Batch 6/66, Loss: 1.2346, Time: 0.83 seconds
Epoch 1/4, Batch 7/66, Loss: 1.2481, Time: 0.83 seconds
Epoch 1/4, Batch 8/66, Loss: 1.1154, Time: 0.83 seconds
Epoch 1/4, Batch 9/66, Loss: 0.9139, Time: 0.83 seconds
Epoch 1/4, Batch 10/66, Loss: 0.8973, Time: 0.83 seconds
Epoch 1/4, Batch 11/66, Loss: 1.4625, Time: 0.83 seconds
Epoch 1/4, Batch 12/66, Loss: 1.3076, Time: 0.83 seconds
Epoch 1/4, Batch 13/66, Loss: 0.8458, Time: 0.83 seconds
Epoch 1/4, Batch 14/66, Loss: 1.2025, Time: 0.83 seconds
Epoch 1/4, Batch 15/66, Loss: 1.3124, Time: 0.83 seconds
Epoch 1/4, Batch 16/66, Loss: 1.4255, Time: 0.83 seconds
Epoch 1/4, Batch 17/66, Loss: 0.8189, Time: 0.83 seconds
Epoch 1/4, Batch 18/66, Loss: 1.0701, Ti

In [ ]:
# model.save_pretrained("./trained_models/UsedClassWeights")
M.save_pretrained("./trained_models/tmp")

NameError: ignored

### Using over-sampling

In [ ]:
!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
# Separate minority and majority classes
minority_classes = ['NetworkDisconnection', 'DiskFull']
minority_df = df[df['RootCause'].isin(minority_classes)]
majority_df = df[~df['RootCause'].isin(minority_classes)]

# Split the dataset into train and validation sets
train_minority, val_minority = train_test_split(minority_df, test_size=0.2, random_state=432)
train_majority, val_majority = train_test_split(majority_df, test_size=0.2, random_state=432)

# Combine minority and majority datasets for training
train_df = pd.concat([train_minority, train_majority], ignore_index=True)

# Apply oversampling to minority classes
oversampler = RandomOverSampler(random_state=432)
X_resampled, y_resampled = oversampler.fit_resample(
    train_df['LogContent'].values.reshape(-1, 1),
    train_df['labels']
)

# Create a new DataFrame with oversampled data
oversampled_df = pd.DataFrame({'LogContent': X_resampled.flatten(), 'labels': y_resampled})

# Combine minority and majority datasets for validation
val_df = pd.concat([val_minority, val_majority], ignore_index=True)

In [ ]:
from collections import Counter

# Assuming 'oversampled_df' is your oversampled DataFrame
class_counts = Counter(oversampled_df['labels'])

# Print the counts for each class
for class_idx, count in class_counts.items():
    print(f"Class {class_idx}: {count} samples")


Class 0: 343 samples
Class 1: 343 samples
Class 2: 343 samples


In [ ]:
# Create CustomDataset instances for training and validation
train_dataset = CustomDataset(texts=list(oversampled_df['LogContent']), labels=oversampled_df['labels'], tokenizer=tokenizer)
val_dataset = CustomDataset(texts=list(val_df['LogContent']), labels=val_df['labels'], tokenizer=tokenizer)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=True)

len(train_dataloader)

258

In [ ]:
# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=NUM_LABELS
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()
device

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device(type='cuda')

In [ ]:
import time

# Training loop
num_epochs = 4
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    start_time = time.time()

    # Iterate over batches in the training DataLoader
    for batch_idx, batch in enumerate(train_dataloader):
        batch_start_time = time.time()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # Print training progress for each batch
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

    average_loss = total_loss / len(train_dataloader)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - start_time

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")

    # Validation
    model.eval()
    val_loss = 0.0
    correct_preds = 0

    # Iterate over batches in the validation DataLoader
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Calculate accuracy
            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += (preds == labels).sum().item()

    average_val_loss = val_loss / len(val_dataloader)
    accuracy = correct_preds / len(val_dataset)
    print(f"Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}")


<ipython-input-84-db7b66662265>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/4, Batch 1/258, Loss: 1.0671, Time: 0.49 seconds
Epoch 1/4, Batch 2/258, Loss: 1.0458, Time: 0.40 seconds
Epoch 1/4, Batch 3/258, Loss: 1.2766, Time: 0.42 seconds
Epoch 1/4, Batch 4/258, Loss: 1.3021, Time: 0.42 seconds
Epoch 1/4, Batch 5/258, Loss: 1.1273, Time: 0.41 seconds
Epoch 1/4, Batch 6/258, Loss: 1.0619, Time: 0.42 seconds
Epoch 1/4, Batch 7/258, Loss: 1.0128, Time: 0.41 seconds
Epoch 1/4, Batch 8/258, Loss: 1.1526, Time: 0.42 seconds
Epoch 1/4, Batch 9/258, Loss: 0.8608, Time: 0.41 seconds
Epoch 1/4, Batch 10/258, Loss: 1.2940, Time: 0.42 seconds
Epoch 1/4, Batch 11/258, Loss: 1.2056, Time: 0.42 seconds
Epoch 1/4, Batch 12/258, Loss: 1.1798, Time: 0.42 seconds
Epoch 1/4, Batch 13/258, Loss: 1.4408, Time: 0.43 seconds
Epoch 1/4, Batch 14/258, Loss: 1.1107, Time: 0.42 seconds
Epoch 1/4, Batch 15/258, Loss: 1.0637, Time: 0.42 seconds
Epoch 1/4, Batch 16/258, Loss: 1.1903, Time: 0.42 seconds
Epoch 1/4, Batch 17/258, Loss: 1.1359, Time: 0.41 seconds
Epoch 1/4, Batch 18/258

In [ ]:
model.save_pretrained("./trained_models/old_way_1")

### Using under-sampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler


# Separate minority and majority classes
majority_class = ['NetworkDisconnection', 'MachineDown']
majority_df = df[df['RootCause'].isin(majority_class)]
minority_df = df[~df['RootCause'].isin(majority_class)]

# Split the dataset into train and validation sets
train_minority, val_minority = train_test_split(minority_df, test_size=0.2, random_state=412)
train_majority, val_majority = train_test_split(majority_df, test_size=0.2, random_state=412)

# Combine minority and majority datasets for training
train_df = pd.concat([train_minority, train_majority], ignore_index=True)


min_samples_per_class = min(train_df['RootCause'].value_counts())
desired_samples = {0: min_samples_per_class, 1: min_samples_per_class, 2: min_samples_per_class}


# Apply undersampling to majority class
undersampler = RandomUnderSampler(sampling_strategy=desired_samples, random_state=412)
X_resampled, y_resampled = undersampler.fit_resample(
    train_df['LogContent'].values.reshape(-1, 1),
    train_df['labels']
)

# Create a new DataFrame with undersampled data
undersampled_df = pd.DataFrame({'LogContent': X_resampled.flatten(), 'labels': y_resampled})

# Combine minority and majority datasets for validation
val_df = pd.concat([val_minority, val_majority], ignore_index=True)


# Apply undersampling to VAL df

min_samples_per_class = min(val_df['RootCause'].value_counts())
desired_samples_val = {0: min_samples_per_class, 1: min_samples_per_class, 2: min_samples_per_class}


undersampler = RandomUnderSampler(sampling_strategy=desired_samples_val, random_state=412)
X_resampled, y_resampled = undersampler.fit_resample(
    val_df['LogContent'].values.reshape(-1, 1),
    val_df['labels']
)

# Create a new DataFrame with undersampled data
val_df = pd.DataFrame({'LogContent': X_resampled.flatten(), 'labels': y_resampled})



In [ ]:
undersampled_df['labels'].value_counts()

0    80
1    80
2    80
Name: labels, dtype: int64

In [ ]:
# Create CustomDataset instances for training and validation
train_dataset = CustomDataset(texts=list(undersampled_df['LogContent']), labels=undersampled_df['labels'], tokenizer=tokenizer)
val_dataset = CustomDataset(texts=list(val_df['LogContent']), labels=val_df['labels'], tokenizer=tokenizer)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=True)

len(train_dataloader)

60

In [ ]:
# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=NUM_LABELS
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()
device

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device(type='cuda')

In [ ]:
import time

# Training loop
num_epochs = 4
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    start_time = time.time()

    # Iterate over batches in the training DataLoader
    for batch_idx, batch in enumerate(train_dataloader):
        batch_start_time = time.time()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # Print training progress for each batch
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

    average_loss = total_loss / len(train_dataloader)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - start_time

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")

    # Validation
    model.eval()
    val_loss = 0.0
    correct_preds = 0

    # Iterate over batches in the validation DataLoader
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Calculate accuracy
            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += (preds == labels).sum().item()

    average_val_loss = val_loss / len(val_dataloader)
    accuracy = correct_preds / len(val_dataset)
    print(f"Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}")


<ipython-input-53-db7b66662265>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/4, Batch 1/60, Loss: 0.9407, Time: 0.46 seconds
Epoch 1/4, Batch 2/60, Loss: 1.1940, Time: 0.41 seconds
Epoch 1/4, Batch 3/60, Loss: 1.6561, Time: 0.43 seconds
Epoch 1/4, Batch 4/60, Loss: 1.1386, Time: 0.41 seconds
Epoch 1/4, Batch 5/60, Loss: 1.0190, Time: 0.42 seconds
Epoch 1/4, Batch 6/60, Loss: 1.2034, Time: 0.42 seconds
Epoch 1/4, Batch 7/60, Loss: 1.1201, Time: 0.42 seconds
Epoch 1/4, Batch 8/60, Loss: 1.0635, Time: 0.42 seconds
Epoch 1/4, Batch 9/60, Loss: 1.3102, Time: 0.42 seconds
Epoch 1/4, Batch 10/60, Loss: 1.0991, Time: 0.42 seconds
Epoch 1/4, Batch 11/60, Loss: 0.9371, Time: 0.43 seconds
Epoch 1/4, Batch 12/60, Loss: 1.0779, Time: 0.42 seconds
Epoch 1/4, Batch 13/60, Loss: 1.2841, Time: 0.42 seconds
Epoch 1/4, Batch 14/60, Loss: 1.3493, Time: 0.43 seconds
Epoch 1/4, Batch 15/60, Loss: 1.1029, Time: 0.43 seconds
Epoch 1/4, Batch 16/60, Loss: 0.9751, Time: 0.42 seconds
Epoch 1/4, Batch 17/60, Loss: 1.1627, Time: 0.43 seconds
Epoch 1/4, Batch 18/60, Loss: 1.0339, Ti

## Using trained model

In [ ]:
# Load the trained model
# model_path = "./trained_models/Batch_8_BERT_Base_acc75"
# model_path = "./trained_models/UsedClassWeights"
# model_path = "./trained_models/old_way"
model_path = "./trained_models/semi_supervised/third_semi_unbalanced"
loaded_model = BertForSequenceClassification.from_pretrained(model_path)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

### PREDICT

In [ ]:
def predict(text, model, tokenizer, device):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to the same device as the model
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=1)
    predicted_class = torch.argmax(probs, dim=1).item()
    return predicted_class, probs[0][predicted_class].item()

# Example usage
# text_to_predict = "Your input text here."
text_to_predict = pseudo_labeled['LogContent'][3000]
predicted_class, confidence = predict(text_to_predict, loaded_model, tokenizer, device)
print(f"Predicted Class: {predicted_class}, Confidence: {confidence:.4f}")

print(pseudo_labeled['labels'][3000])


Predicted Class: 2, Confidence: 0.6677
2


In [ ]:
loaded_model.eval()
# model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_dataloader:
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = loaded_model(input_ids, attention_mask=attention_mask)
        # outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds))


<ipython-input-50-33f4ade4f096>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


              precision    recall  f1-score   support

           0       0.96      0.58      0.73       177
           1       0.63      0.48      0.55       156
           2       0.80      0.94      0.87       640

    accuracy                           0.80       973
   macro avg       0.80      0.67      0.71       973
weighted avg       0.81      0.80      0.79       973



In [ ]:
# Assuming df is your DataFrame
logs_with_label_0 = df[df['labels'] == 0]

# Display LogContents where label is 0
print(logs_with_label_0['LogContent'][0])


   metrics2.impl.MetricsConfig: loaded properties from hadoop-metrics2.properties
   metrics2.impl.MetricsSystemImpl: Scheduled snapshot period at 10 second(s).
   metrics2.impl.MetricsSystemImpl: MapTask metrics system started
   mapred.YarnChild: Executing with tokens:
   mapred.YarnChild: Kind: mapreduce.job, Service: job_1445144423722_0022, Ident: (mapreduce.security.token.JobTokenIdentifier@6ace4625)
   mapred.YarnChild: Sleeping for 0ms before retrying again. Got null now.
   mapred.YarnChild: mapreduce.cluster.local.dir for child: /tmp/hadoop-msrabi/nm-local-dir/usercache/msrabi/appcache/application_1445144423722_0022
   conf.Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
   yarn.util.ProcfsBasedProcessTree: ProcfsBasedProcessTree currently is supported only on Linux.
   mapred.Task:  Using ResourceCalculatorProcessTree : yarn.util.WindowsBasedProcessTree@3c367995
   mapred.MapTask: Processing split: hdfs://msra-sa-41:9000/pageinput2.txt

# Semi Supervised

In [ ]:
chunked = pd.read_csv('forClassify_2.csv')
original = pd.read_csv('forClassify_1.csv')
# df.columns
del original['Unnamed: 0.2']
del original['Unnamed: 0.1']
del original['Unnamed: 0']
del original['tokens']
del original['FileName']
del chunked['token_numbers']
del chunked['Unnamed: 0']
# df

In [ ]:
df = pd.concat([chunked, original], ignore_index=True)
labels = df['RootCause'].unique().tolist()
labels = [s.strip() for s in labels ]
NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}
df["labels"]=df.RootCause.map(lambda x: label2id[x.strip()])
df

,LogContent,RootCause,labels
0,metrics2.impl.\n\nMetricsConfig: loaded proper...,NetworkDisconnection,0
1,MapTask: (EQUATOR) 0\n\nkvi 26214396(104857584...,NetworkDisconnection,0
2,MapTask: (EQUATOR) 9726918\n\nkvi 2431724(9726...,NetworkDisconnection,0
3,metrics2.impl.\n\nMetricsConfig: loaded proper...,NetworkDisconnection,0
4,MapTask: (EQUATOR) 0\n\nkvi 26214396(104857584...,NetworkDisconnection,0
...,...,...,...
5517,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,2
5518,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,2
5519,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,2
5520,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,2


In [ ]:
# Split the data into labeled and unlabeled datasets (80-20 split)
labeled, unlabeled = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
labeled = labeled.reset_index(drop=True)
unlabeled = unlabeled.reset_index(drop=True)
labeled['labels'].value_counts()
# labeled

2    2938
0     775
1     704
Name: labels, dtype: int64

In [ ]:
labeled_dataset = CustomDataset(texts=list(labeled['LogContent']), labels=labeled['labels'], tokenizer=tokenizer)

# Create DataLoader for labeled dataset
labeled_dataloader = DataLoader(labeled_dataset, batch_size=16, shuffle=True)

In [ ]:
# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=NUM_LABELS
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import time

# Assuming labeled_train_dataloader is your DataLoader for labeled training data

# Fine-tuning loop
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    start_time = time.time()

    # Iterate over batches in the labeled training DataLoader
    for batch_idx, batch in enumerate(labeled_dataloader):
        batch_start_time = time.time()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Print label counts for the current batch
        label_counts = {label: count for label, count in zip(*np.unique(labels.cpu().numpy(), return_counts=True))}
        print(f"Batch {batch_idx + 1}/{len(labeled_dataloader)}, Label Counts: {label_counts}")

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # Print training progress for each batch
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(labeled_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

    average_loss = total_loss / len(labeled_dataloader)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - start_time

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")


<ipython-input-50-33f4ade4f096>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Batch 1/277, Label Counts: {0: 1, 1: 1, 2: 14}
Epoch 1/2, Batch 1/277, Loss: 1.5977, Time: 1.40 seconds
Batch 2/277, Label Counts: {0: 3, 1: 3, 2: 10}
Epoch 1/2, Batch 2/277, Loss: 1.0071, Time: 1.33 seconds
Batch 3/277, Label Counts: {0: 3, 1: 4, 2: 9}
Epoch 1/2, Batch 3/277, Loss: 0.9736, Time: 1.35 seconds
Batch 4/277, Label Counts: {0: 3, 1: 4, 2: 9}
Epoch 1/2, Batch 4/277, Loss: 0.9710, Time: 1.35 seconds
Batch 5/277, Label Counts: {0: 2, 1: 4, 2: 10}
Epoch 1/2, Batch 5/277, Loss: 0.8982, Time: 1.35 seconds
Batch 6/277, Label Counts: {0: 4, 1: 4, 2: 8}
Epoch 1/2, Batch 6/277, Loss: 1.1257, Time: 1.35 seconds
Batch 7/277, Label Counts: {0: 3, 2: 13}
Epoch 1/2, Batch 7/277, Loss: 0.7823, Time: 1.35 seconds
Batch 8/277, Label Counts: {0: 2, 1: 5, 2: 9}
Epoch 1/2, Batch 8/277, Loss: 0.9698, Time: 1.35 seconds
Batch 9/277, Label Counts: {0: 3, 1: 3, 2: 10}
Epoch 1/2, Batch 9/277, Loss: 0.9372, Time: 1.35 seconds
Batch 10/277, Label Counts: {0: 4, 1: 3, 2: 9}
Epoch 1/2, Batch 10/277, Lo

In [ ]:
model.save_pretrained("./trained_models/semi_supervised/fine_tuned_3")

In [ ]:
# %cd ..
# %cd trained_models/semi_supervised
# unlabeled.to_csv('unlabeled.csv')
# labeled.to_csv('labeled.csv')

/content/gdrive/MyDrive/MHA/Project


In [ ]:
# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

### Predict labels using the fine-tuned model

In [ ]:
unlabeled

,LogContent,RootCause,labels
0,MapTask: (EQUATOR) 0\n\nkvi 26214396(104857584...,MachineDown,2
1,metrics2.impl.\n\nMetricsConfig: loaded proper...,MachineDown,2
2,TaskAttemptListenerImpl: Progress of TaskAttem...,MachineDown,2
3,metrics2.impl.\n\nMetricsConfig: loaded proper...,MachineDown,2
4,metrics2.impl.\n\nMetricsConfig: loaded proper...,MachineDown,2
...,...,...,...
1100,MapTask: (EQUATOR) 0\n\nkvi 26214396(104857584...,MachineDown,2
1101,MapTask: Processing split: hdfs://msra-sa-41:9...,MachineDown,2
1102,ShuffleSchedulerImpl: assigned 1 of 1 to MSRA-...,MachineDown,2
1103,metrics2.impl.\n\nMetricsConfig: loaded proper...,MachineDown,2


In [ ]:
#load model and unlabeled dataset if needed
# model_path = "./trained_models/old_way"
# model = BertForSequenceClassification.from_pretrained(model_path)

# labeled = pd.read_csv('./trained_models/semi_supervised/labeled.csv')
# unlabeled = pd.read_csv('./trained_models/semi_supervised/unlabeled.csv')

# Move the model to the same device as the training
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Assuming 'unlabeled' is your unlabeled dataset
batch_size = 8  # Adjust the batch size based on your available GPU memory

# Tokenize the unlabeled texts
unlabeled_texts = unlabeled['LogContent'].tolist()
num_batches = int(np.ceil(len(unlabeled_texts) / batch_size))

predicted_labels = []

# Make predictions batch-wise
with torch.no_grad():
    model.eval()
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(unlabeled_texts))

        batch_texts = unlabeled_texts[start_idx:end_idx]

        # Tokenize the batch
        batch_encodings = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
        batch_inputs = {key: val.to(device) for key, val in batch_encodings.items()}

        # Make predictions for the batch
        outputs = model(**batch_inputs)
        batch_predictions = torch.argmax(outputs.logits, dim=1)

        predicted_labels.extend(batch_predictions.cpu().numpy())

# Create a new dataframe with the original columns and the predicted labels
predicted_df = pd.DataFrame({
    'LogContent': unlabeled['LogContent'],
    'RootCause': unlabeled['RootCause'],
    'PredictedLabels': predicted_labels
})

predicted_df = predicted_df.rename(columns={'PredictedLabels': 'labels'})

predicted_df

,LogContent,RootCause,labels
0,MapTask: Spilling map output\n mapred.\n\nMa...,DiskFull,2
1,ContainerManagementProtocolProxy:\n\nOpening p...,MachineDown,2
2,ShuffleSchedulerImpl: assigned 5 of 5 to MSRA-...,MachineDown,2
3,metrics2.impl.\n\nMetricsConfig: loaded proper...,MachineDown,1
4,TaskAttemptImpl: TaskAttempt: [attempt_1445062...,MachineDown,2
...,...,...,...
1100,MapTask: (EQUATOR) 89325783 kvi 22331440(89325...,MachineDown,2
1101,MapTask: (EQUATOR) 9764656 kvi 2441160(9764640...,MachineDown,2
1102,metrics2.impl.\n\nMetricsConfig: loaded proper...,DiskFull,1
1103,"MergeManagerImpl: Merging 13 files, 2831866878...",MachineDown,0


In [ ]:
predicted_df['labels'].value_counts()

2    813
1    156
0    136
Name: labels, dtype: int64

In [ ]:
unlabeled['labels'].value_counts()

2    706
0    214
1    185
Name: labels, dtype: int64

In [ ]:
pseudo_labeled = pd.concat([labeled, predicted_df], ignore_index=True)

In [ ]:
pseudo_labeled['labels'].value_counts()

2    3751
0     911
1     860
Name: labels, dtype: int64

In [ ]:
# Split the data into train and validation datasets (80-20 split)
train, val = train_test_split(pseudo_labeled, test_size=0.2, random_state=42, shuffle=True)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

In [ ]:
# Create CustomDataset instances for training and validation
train_dataset = CustomDataset(texts=list(train['LogContent']), labels=train['labels'], tokenizer=tokenizer)
val_dataset = CustomDataset(texts=list(val['LogContent']), labels=val['labels'], tokenizer=tokenizer)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)

len(train_dataloader)

553

In [ ]:
import time
# Training loop without early stopping
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    start_time = time.time()

    # Iterate over batches in the training DataLoader
    for batch_idx, batch in enumerate(train_dataloader):
        batch_start_time = time.time()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Print label counts for the current batch
        label_counts = {label: count for label, count in zip(*np.unique(labels.cpu().numpy(), return_counts=True))}
        print(f"Batch {batch_idx + 1}/{len(train_dataloader)}, Label Counts: {label_counts}")

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # Print training progress for each batch
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

    average_loss = total_loss / len(train_dataloader)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - start_time

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")

    # Validation
    model.eval()
    val_loss = 0.0
    correct_preds = 0

    # Iterate over batches in the validation DataLoader
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Calculate accuracy
            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += (preds == labels).sum().item()

    average_val_loss = val_loss / len(val_dataloader)
    accuracy = correct_preds / len(val_dataset)
    print(f"Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}")


<ipython-input-50-33f4ade4f096>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/10, Batch 1/553, Loss: 0.3085, Time: 0.73 seconds
Epoch 1/10, Batch 2/553, Loss: 0.3919, Time: 0.68 seconds
Epoch 1/10, Batch 3/553, Loss: 0.3218, Time: 0.67 seconds
Epoch 1/10, Batch 4/553, Loss: 0.4004, Time: 0.67 seconds
Epoch 1/10, Batch 5/553, Loss: 0.5880, Time: 0.68 seconds
Epoch 1/10, Batch 6/553, Loss: 0.8048, Time: 0.68 seconds
Epoch 1/10, Batch 7/553, Loss: 0.2975, Time: 0.68 seconds
Epoch 1/10, Batch 8/553, Loss: 0.3838, Time: 0.69 seconds
Epoch 1/10, Batch 9/553, Loss: 0.4310, Time: 0.68 seconds
Epoch 1/10, Batch 10/553, Loss: 0.5555, Time: 0.68 seconds
Epoch 1/10, Batch 11/553, Loss: 0.4149, Time: 0.69 seconds
Epoch 1/10, Batch 12/553, Loss: 0.3717, Time: 0.68 seconds
Epoch 1/10, Batch 13/553, Loss: 0.3128, Time: 0.68 seconds
Epoch 1/10, Batch 14/553, Loss: 0.2969, Time: 0.67 seconds
Epoch 1/10, Batch 15/553, Loss: 0.4341, Time: 0.69 seconds
Epoch 1/10, Batch 16/553, Loss: 0.5791, Time: 0.69 seconds
Epoch 1/10, Batch 17/553, Loss: 0.1580, Time: 0.69 seconds
Epoch 

KeyboardInterrupt: ignored

In [ ]:
#training loop with early stopping

# Set the initial best validation loss to a large value
best_val_loss = float('inf')
best_model_state_dict = model.state_dict()
patience = 3  # Number of epochs with no improvement after which training will be stopped due to overfitting

# Training loop
num_epochs = 20  # Adjust the number of epochs as needed
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    start_time = time.time()

    # Iterate over batches in the training DataLoader
    for batch_idx, batch in enumerate(train_dataloader):
        batch_start_time = time.time()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # Print training progress for each batch
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

    average_loss = total_loss / len(train_dataloader)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - start_time

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")

    # Validation
    model.eval()
    val_loss = 0.0
    correct_preds = 0

    # Iterate over batches in the validation DataLoader
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Calculate accuracy
            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += (preds == labels).sum().item()

    average_val_loss = val_loss / len(val_dataloader)
    accuracy = correct_preds / len(val_dataset)
    print(f"Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}")

    # Early stopping check for overfitting
    if epoch > 0 and average_loss < best_val_loss:
        best_val_loss = average_loss
        best_model_state_dict = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Overfitting detected. Early stopping.")
            break

# Load the best model state
model.load_state_dict(best_model_state_dict)


<ipython-input-50-33f4ade4f096>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/20, Batch 1/553, Loss: 0.1735, Time: 0.71 seconds
Epoch 1/20, Batch 2/553, Loss: 0.4088, Time: 0.70 seconds
Epoch 1/20, Batch 3/553, Loss: 0.0602, Time: 0.67 seconds
Epoch 1/20, Batch 4/553, Loss: 0.9496, Time: 0.69 seconds
Epoch 1/20, Batch 5/553, Loss: 0.2855, Time: 0.70 seconds
Epoch 1/20, Batch 6/553, Loss: 0.1677, Time: 0.69 seconds
Epoch 1/20, Batch 7/553, Loss: 0.5995, Time: 0.70 seconds
Epoch 1/20, Batch 8/553, Loss: 0.3286, Time: 0.69 seconds
Epoch 1/20, Batch 9/553, Loss: 0.3679, Time: 0.70 seconds
Epoch 1/20, Batch 10/553, Loss: 0.6956, Time: 0.69 seconds
Epoch 1/20, Batch 11/553, Loss: 0.3047, Time: 0.70 seconds
Epoch 1/20, Batch 12/553, Loss: 0.1594, Time: 0.69 seconds
Epoch 1/20, Batch 13/553, Loss: 0.2976, Time: 0.69 seconds
Epoch 1/20, Batch 14/553, Loss: 0.0576, Time: 0.70 seconds
Epoch 1/20, Batch 15/553, Loss: 0.2904, Time: 0.69 seconds
Epoch 1/20, Batch 16/553, Loss: 0.7614, Time: 0.70 seconds
Epoch 1/20, Batch 17/553, Loss: 0.2403, Time: 0.69 seconds
Epoch 

<All keys matched successfully>

In [ ]:
# model.save_pretrained("./trained_models/semi_supervised/fine_tuned")

In [ ]:
# model.save_pretrained("./trained_models/semi_supervised/first_semi_unbalanced")

In [ ]:
model.save_pretrained("./trained_models/semi_supervised/third_semi_unbalanced")

# Semi Supervised (new way)

In [ ]:
chunked = pd.read_csv('chunked_dataset.csv')
original = pd.read_csv('dataset.csv')
del original['Unnamed: 0']
del original['labels']
del chunked['token_numbers']
del chunked['Unnamed: 0']
df = pd.concat([chunked, original], ignore_index=True)
df

,LogContent,RootCause
0,metrics2.impl.\n\nMetricsConfig: loaded proper...,MachineDown
1,MapTask: Processing split: hdfs://msra-sa-41:9...,MachineDown
2,MapTask: kvstart = 11165416(44661664); kvend =...,MachineDown
3,[SpillThread] mapred.\n\nMapTask: Finished spi...,MachineDown
4,metrics2.impl.\n\nMetricsConfig: loaded proper...,NetworkDisconnection
...,...,...
4795,metrics2.impl.MetricsConfig: loaded propert...,DiskFull
4796,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
4797,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
4798,metrics2.impl.MetricsConfig: loaded propert...,DiskFull


In [ ]:
labels = df['RootCause'].unique().tolist()
labels = [s.strip() for s in labels ]
NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}
df["labels"]=df.RootCause.map(lambda x: label2id[x.strip()])
df

,LogContent,RootCause,labels
0,metrics2.impl.\n\nMetricsConfig: loaded proper...,MachineDown,0
1,MapTask: Processing split: hdfs://msra-sa-41:9...,MachineDown,0
2,MapTask: kvstart = 11165416(44661664); kvend =...,MachineDown,0
3,[SpillThread] mapred.\n\nMapTask: Finished spi...,MachineDown,0
4,metrics2.impl.\n\nMetricsConfig: loaded proper...,NetworkDisconnection,1
...,...,...,...
4795,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1
4796,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
4797,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
4798,metrics2.impl.MetricsConfig: loaded propert...,DiskFull,2


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=512)
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Split the data into labeled and unlabeled datasets (90-10 split)
labeled, unlabeled = train_test_split(df, test_size=0.1, random_state=100, shuffle=True)
labeled = labeled.reset_index(drop=True)
unlabeled = unlabeled.reset_index(drop=True)
labeled['labels'].value_counts()
# labeled

0    2830
1     793
2     697
Name: labels, dtype: int64

In [ ]:
labeled_dataset = CustomDataset(texts=list(labeled['LogContent']), labels=labeled['labels'], tokenizer=tokenizer)

# Create DataLoader for labeled dataset
labeled_dataloader = DataLoader(labeled_dataset, batch_size=16, shuffle=True)

In [ ]:
# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=NUM_LABELS
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import time

# Assuming labeled_train_dataloader is your DataLoader for labeled training data

# Fine-tuning loop
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    start_time = time.time()

    # Iterate over batches in the labeled training DataLoader
    for batch_idx, batch in enumerate(labeled_dataloader):
        batch_start_time = time.time()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Print label counts for the current batch
        # label_counts = {label: count for label, count in zip(*np.unique(labels.cpu().numpy(), return_counts=True))}
        # print(f"Batch {batch_idx + 1}/{len(labeled_dataloader)}, Label Counts: {label_counts}")

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # Print training progress for each batch
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(labeled_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

    average_loss = total_loss / len(labeled_dataloader)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - start_time

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")


<ipython-input-5-ef9e455f6b36>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/2, Batch 1/270, Loss: 1.0872, Time: 2.19 seconds
Epoch 1/2, Batch 2/270, Loss: 0.9036, Time: 1.44 seconds
Epoch 1/2, Batch 3/270, Loss: 1.0897, Time: 1.44 seconds
Epoch 1/2, Batch 4/270, Loss: 1.2179, Time: 1.46 seconds
Epoch 1/2, Batch 5/270, Loss: 1.0850, Time: 1.44 seconds
Epoch 1/2, Batch 6/270, Loss: 0.7738, Time: 1.47 seconds
Epoch 1/2, Batch 7/270, Loss: 0.6794, Time: 1.46 seconds
Epoch 1/2, Batch 8/270, Loss: 0.7063, Time: 1.48 seconds
Epoch 1/2, Batch 9/270, Loss: 0.4248, Time: 1.46 seconds
Epoch 1/2, Batch 10/270, Loss: 0.7465, Time: 1.48 seconds
Epoch 1/2, Batch 11/270, Loss: 0.6434, Time: 1.50 seconds
Epoch 1/2, Batch 12/270, Loss: 0.7231, Time: 1.50 seconds
Epoch 1/2, Batch 13/270, Loss: 0.8990, Time: 1.52 seconds
Epoch 1/2, Batch 14/270, Loss: 1.5212, Time: 1.52 seconds
Epoch 1/2, Batch 15/270, Loss: 0.4809, Time: 1.51 seconds
Epoch 1/2, Batch 16/270, Loss: 1.2654, Time: 1.53 seconds
Epoch 1/2, Batch 17/270, Loss: 0.9324, Time: 1.54 seconds
Epoch 1/2, Batch 18/270

In [ ]:
model.save_pretrained("./trained_models/semi_supervised_new/fine_tuned")

In [ ]:
labeled

,LogContent,RootCause,labels
0,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
1,MapTask: (EQUATOR) 0\n\nkvi 26214396(104857584...,NetworkDisconnection,1
2,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
3,NetUtils.wrapException(NetUtils.java:757)\n\ta...,NetworkDisconnection,1
4,MapTask: (EQUATOR) 9745912 kvi 2436472(9745888...,NetworkDisconnection,1
...,...,...,...
3835,MapTask: (EQUATOR) 9811814 kvi 2452948(9811792...,MachineDown,0
3836,MapTask: kvstart = 14562400(58249600); kvend =...,MachineDown,0
3837,ContainerManagementProtocolProxy:\n\nOpening p...,MachineDown,0
3838,MapTask: (EQUATOR) 0\n\nkvi 26214396(104857584...,MachineDown,0


In [ ]:
#load model and unlabeled dataset if needed
# model_path = "./trained_models/old_way"
# model = BertForSequenceClassification.from_pretrained(model_path)

# labeled = pd.read_csv('./trained_models/semi_supervised/labeled.csv')
# unlabeled = pd.read_csv('./trained_models/semi_supervised/unlabeled.csv')

# Move the model to the same device as the training
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Assuming 'unlabeled' is your unlabeled dataset
batch_size = 8  # Adjust the batch size based on your available GPU memory

# Tokenize the unlabeled texts
unlabeled_texts = unlabeled['LogContent'].tolist()
num_batches = int(np.ceil(len(unlabeled_texts) / batch_size))

predicted_labels = []

# Make predictions batch-wise
with torch.no_grad():
    model.eval()
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(unlabeled_texts))

        batch_texts = unlabeled_texts[start_idx:end_idx]

        # Tokenize the batch
        batch_encodings = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
        batch_inputs = {key: val.to(device) for key, val in batch_encodings.items()}

        # Make predictions for the batch
        outputs = model(**batch_inputs)
        batch_predictions = torch.argmax(outputs.logits, dim=1)

        predicted_labels.extend(batch_predictions.cpu().numpy())

# Create a new dataframe with the original columns and the predicted labels
predicted_df = pd.DataFrame({
    'LogContent': unlabeled['LogContent'],
    'RootCause': unlabeled['RootCause'],
    'PredictedLabels': predicted_labels
})

predicted_df = predicted_df.rename(columns={'PredictedLabels': 'labels'})

predicted_df

,LogContent,RootCause,labels
0,MapTask: Spilling map output\n mapred.\n\nMa...,MachineDown,0
1,MapTask: kvstart = 7315060(29260240); kvend = ...,MachineDown,0
2,ShuffleSchedulerImpl: assigned 1 of 1 to MSRA-...,MachineDown,0
3,MapTask: kvstart = 7315060(29260240); kvend = ...,DiskFull,2
4,MapTask: (EQUATOR) 0\n\nkvi 26214396(104857584...,MachineDown,0
...,...,...,...
475,RackResolver: Resolved MSRA-SA-39.fareast.corp...,MachineDown,0
476,Fetcher: for url=13562/mapOutput?job=job_14451...,DiskFull,2
477,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
478,metrics2.impl.\n\nMetricsConfig: loaded proper...,MachineDown,0


In [ ]:
unlabeled['labels'].value_counts()

0    340
1     83
2     57
Name: labels, dtype: int64

In [ ]:
predicted_df['labels'].value_counts()

0    382
1     58
2     40
Name: labels, dtype: int64

In [ ]:
pseudo_labeled = pd.concat([labeled, predicted_df], ignore_index=True)

In [ ]:
pseudo_labeled['labels'].value_counts()

0    3212
1     851
2     737
Name: labels, dtype: int64

In [ ]:
# Split the data into train and validation datasets (80-20 split)
train, val = train_test_split(pseudo_labeled, test_size=0.2, random_state=100, shuffle=True)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

In [ ]:
# Create CustomDataset instances for training and validation
train_dataset = CustomDataset(texts=list(train['LogContent']), labels=train['labels'], tokenizer=tokenizer)
val_dataset = CustomDataset(texts=list(val['LogContent']), labels=val['labels'], tokenizer=tokenizer)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)

len(train_dataloader)

480

In [ ]:
import time
# Training loop without early stopping
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    start_time = time.time()

    # Iterate over batches in the training DataLoader
    for batch_idx, batch in enumerate(train_dataloader):
        batch_start_time = time.time()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Print label counts for the current batch
        label_counts = {label: count for label, count in zip(*np.unique(labels.cpu().numpy(), return_counts=True))}
        print(f"Batch {batch_idx + 1}/{len(train_dataloader)}, Label Counts: {label_counts}")

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # Print training progress for each batch
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

    average_loss = total_loss / len(train_dataloader)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - start_time

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")

    # Validation
    model.eval()
    val_loss = 0.0
    correct_preds = 0

    # Iterate over batches in the validation DataLoader
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Calculate accuracy
            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += (preds == labels).sum().item()

    average_val_loss = val_loss / len(val_dataloader)
    accuracy = correct_preds / len(val_dataset)
    print(f"Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}")


<ipython-input-5-ef9e455f6b36>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Batch 1/480, Label Counts: {0: 5, 1: 3}
Epoch 1/3, Batch 1/480, Loss: 0.4445, Time: 0.80 seconds
Batch 2/480, Label Counts: {0: 5, 1: 2, 2: 1}
Epoch 1/3, Batch 2/480, Loss: 0.2836, Time: 0.79 seconds
Batch 3/480, Label Counts: {0: 6, 1: 1, 2: 1}
Epoch 1/3, Batch 3/480, Loss: 0.3159, Time: 0.79 seconds
Batch 4/480, Label Counts: {0: 5, 2: 3}
Epoch 1/3, Batch 4/480, Loss: 0.2976, Time: 0.78 seconds
Batch 5/480, Label Counts: {0: 5, 2: 3}
Epoch 1/3, Batch 5/480, Loss: 0.5347, Time: 0.78 seconds
Batch 6/480, Label Counts: {0: 6, 1: 2}
Epoch 1/3, Batch 6/480, Loss: 0.2952, Time: 0.78 seconds
Batch 7/480, Label Counts: {0: 7, 2: 1}
Epoch 1/3, Batch 7/480, Loss: 0.2000, Time: 0.79 seconds
Batch 8/480, Label Counts: {0: 3, 1: 4, 2: 1}
Epoch 1/3, Batch 8/480, Loss: 0.7093, Time: 0.80 seconds
Batch 9/480, Label Counts: {0: 6, 1: 2}
Epoch 1/3, Batch 9/480, Loss: 0.1332, Time: 0.79 seconds
Batch 10/480, Label Counts: {0: 3, 1: 3, 2: 2}
Epoch 1/3, Batch 10/480, Loss: 0.6154, Time: 0.79 seconds
Batc

In [ ]:
model.save_pretrained("./trained_models/semi_supervised_new/semi_new_1")

### **Evaluation**

In [ ]:
eval = pd.read_csv('evaluation.csv')
del eval['Unnamed: 0']
del eval['labels']
eval

,LogContent,RootCause
0,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
1,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
2,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
3,metrics2.impl.MetricsConfig: loaded propert...,DiskFull
4,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection
...,...,...
127,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
128,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
129,metrics2.impl.MetricsConfig: loaded propert...,DiskFull
130,metrics2.impl.MetricsConfig: loaded propert...,MachineDown


In [ ]:
# Define the mapping for label names to numerical IDs
label_mapping = {
    'MachineDown': 0,
    'NetworkDisconnection': 1,
    'DiskFull': 2
}

# Set the 'labels' column based on the mapping
eval['labels'] = eval['RootCause'].map(lambda x: label_mapping[x.strip()])

# Create dictionaries for label mappings
id2label = {id: label for label, id in label_mapping.items()}
label2id = {label: id for label, id in label_mapping.items()}
eval

,LogContent,RootCause,labels
0,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
1,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
2,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
3,metrics2.impl.MetricsConfig: loaded propert...,DiskFull,2
4,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1
...,...,...,...
127,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
128,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
129,metrics2.impl.MetricsConfig: loaded propert...,DiskFull,2
130,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0


In [ ]:
evaluation_dataset = CustomDataset(texts=list(eval['LogContent']), labels=eval['labels'], tokenizer=tokenizer)

# Create DataLoader for eval dataset
evaluation_dataloader = DataLoader(evaluation_dataset, batch_size=16, shuffle=True)

TypeError: CustomDataset.__init__() missing 1 required positional argument: 'related_ids'

In [ ]:
# loaded_model.eval()
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in evaluation_dataloader:
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # outputs = loaded_model(input_ids, attention_mask=attention_mask)
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds))


# Chunking

In [ ]:
def chunk_text_with_overlap(text, tokenizer, chunk_size=512, overlap_percentage=30):
    # Tokenize the input text
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))

    # Exclude [CLS] and [SEP] tokens
    tokens = [token for token in tokens if token not in ['[CLS]', '[SEP]']]

    # Calculate the overlap in tokens
    overlap_tokens = int(chunk_size * overlap_percentage / 100)

    # Chunk the tokens with overlap
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size - overlap_tokens)]

    # Convert token chunks back to text chunks
    text_chunks = [tokenizer.decode(tokenizer.convert_tokens_to_ids(chunk)) for chunk in chunks]

    return text_chunks

In [ ]:
large_text = "Hello it's a large text for example"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Chunking based on tokens with 20% overlap
chunks = chunk_text_with_overlap(large_text, tokenizer, chunk_size=2, overlap_percentage=50)

# Print tokenized chunks
for i, chunk in enumerate(chunks, 1):
    print(f"Chunk {i}:", chunk)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Chunk 1: hello it
Chunk 2: it '
Chunk 3: ' s
Chunk 4: s a
Chunk 5: a large
Chunk 6: large text
Chunk 7: text for
Chunk 8: for example
Chunk 9: example


In [ ]:
#new chunking
# df_1 = pd.read_csv('dataset.csv')
# df_2 = pd.read_csv('largerLogs.csv')
# data_process = pd.concat([df_1, df_2], ignore_index=True)

data_process = pd.read_csv('dataset.csv')
del data_process['Unnamed: 0']
data_process

,LogContent,RootCause
0,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
1,metrics2.impl.MetricsConfig: loaded propert...,DiskFull
2,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
3,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection
4,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
...,...,...
601,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
602,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
603,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
604,metrics2.impl.MetricsConfig: loaded propert...,MachineDown


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
data_process['TextChunks'] = data_process['LogContent'].apply(lambda x: chunk_text_with_overlap(x, tokenizer, chunk_size=508, overlap_percentage=30))

Token indices sequence length is longer than the specified maximum sequence length for this model (1843 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
chunked_logs = pd.DataFrame(columns=['LogContent', 'RootCause'])
tmp = []
for index, row in data_process.iterrows():
    root_cause = row['RootCause']
    log_chunks = row['TextChunks']

    # Create a DataFrame for the current row
    df = pd.DataFrame({'LogContent': log_chunks, 'RootCause': [root_cause] * len(log_chunks)})

    # Append to the list
    tmp.append(df)

# Concatenate the list of DataFrames into a single DataFrame
chunked_logs = pd.concat(tmp, ignore_index=True)

# Reorder the columns
chunked_logs = chunked_logs[['LogContent', 'RootCause']]

In [ ]:
# Apply tokenizer.convert_ids_to_tokens and calculate the length of tokens
chunked_logs['token_numbers'] = chunked_logs['LogContent'].apply(lambda x: len(tokenizer.convert_ids_to_tokens(tokenizer.encode(x))))

In [ ]:
chunked_logs

,LogContent,RootCause,token_numbers
0,metrics2. impl. metricsconfig : loaded propert...,MachineDown,510
1,. maptask : ( equator ) 0 kvi 26214396 ( 10485...,MachineDown,510
2,##k : kvstart = 11165712 ( 44662848 ) ; kvend ...,MachineDown,512
3,: kvstart = 7281300 ( 29125200 ) ; kvend = 210...,MachineDown,510
4,##388 ( 13585552 ) ; kvend = 17183192 ( 687327...,MachineDown,421
...,...,...,...
4369,maptask : ( equator ) 0 kvi 26214396 ( 1048575...,MachineDown,510
4370,kvstart = 14330776 ( 57323104 ) ; kvend = 5417...,MachineDown,510
4371,##t = 16762552 ( 67050208 ) ; kvend = 7853256 ...,MachineDown,512
4372,##4 ( 76802656 ) ; kvend = 10290256 ( 41161024...,MachineDown,512


In [ ]:
# chunked_logs[chunked_logs['token_numbers']<100].count()
chunked_logs['token_numbers'].max()

513

In [ ]:
# del chunked_logs['token_numbers']
chunked_logs.to_csv('chunked_new.csv')

### **training**

In [ ]:
chunked_logs = pd.read_csv('chunked_new.csv')
del chunked_logs['Unnamed: 0']
chunked_logs

,LogContent,RootCause
0,metrics2. impl. metricsconfig : loaded propert...,MachineDown
1,. maptask : ( equator ) 0 kvi 26214396 ( 10485...,MachineDown
2,##k : kvstart = 11165712 ( 44662848 ) ; kvend ...,MachineDown
3,: kvstart = 7281300 ( 29125200 ) ; kvend = 210...,MachineDown
4,##388 ( 13585552 ) ; kvend = 17183192 ( 687327...,MachineDown
...,...,...
4369,maptask : ( equator ) 0 kvi 26214396 ( 1048575...,MachineDown
4370,kvstart = 14330776 ( 57323104 ) ; kvend = 5417...,MachineDown
4371,##t = 16762552 ( 67050208 ) ; kvend = 7853256 ...,MachineDown
4372,##4 ( 76802656 ) ; kvend = 10290256 ( 41161024...,MachineDown


In [ ]:
NUM_LABELS= 3

# Define the mapping for label names to numerical IDs
label_mapping = {
    'MachineDown': 0,
    'NetworkDisconnection': 1,
    'DiskFull': 2
}

# Set the 'labels' column based on the mapping
chunked_logs['labels'] = chunked_logs['RootCause'].map(lambda x: label_mapping[x.strip()])

# Create dictionaries for label mappings
id2label = {id: label for label, id in label_mapping.items()}
label2id = {label: id for label, id in label_mapping.items()}
chunked_logs

,LogContent,RootCause,labels
0,metrics2. impl. metricsconfig : loaded propert...,MachineDown,0
1,. maptask : ( equator ) 0 kvi 26214396 ( 10485...,MachineDown,0
2,##k : kvstart = 11165712 ( 44662848 ) ; kvend ...,MachineDown,0
3,: kvstart = 7281300 ( 29125200 ) ; kvend = 210...,MachineDown,0
4,##388 ( 13585552 ) ; kvend = 17183192 ( 687327...,MachineDown,0
...,...,...,...
4369,maptask : ( equator ) 0 kvi 26214396 ( 1048575...,MachineDown,0
4370,kvstart = 14330776 ( 57323104 ) ; kvend = 5417...,MachineDown,0
4371,##t = 16762552 ( 67050208 ) ; kvend = 7853256 ...,MachineDown,0
4372,##4 ( 76802656 ) ; kvend = 10290256 ( 41161024...,MachineDown,0


In [ ]:
chunked_logs['labels'].value_counts()

0    2975
1     755
2     644
Name: labels, dtype: int64

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=512)
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Split the data into train and validation datasets (80-20 split)
train, val = train_test_split(chunked_logs, test_size=0.2, random_state=45, shuffle=True)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

In [ ]:
# Create CustomDataset instances for training and validation
train_dataset = CustomDataset(texts=list(train['LogContent']), labels=train['labels'], tokenizer=tokenizer)
val_dataset = CustomDataset(texts=list(val['LogContent']), labels=val['labels'], tokenizer=tokenizer)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [ ]:
# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=NUM_LABELS
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()
device

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device(type='cuda')

In [ ]:
import time
# Training loop without early stopping
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    start_time = time.time()

    # Iterate over batches in the training DataLoader
    for batch_idx, batch in enumerate(train_dataloader):
        batch_start_time = time.time()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Print label counts for the current batch
        label_counts = {label: count for label, count in zip(*np.unique(labels.cpu().numpy(), return_counts=True))}
        print(f"Batch {batch_idx + 1}/{len(train_dataloader)}, Label Counts: {label_counts}")

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # Print training progress for each batch
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

    average_loss = total_loss / len(train_dataloader)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - start_time

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")

    # Validation
    model.eval()
    val_loss = 0.0
    correct_preds = 0

    # Iterate over batches in the validation DataLoader
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Calculate accuracy
            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += (preds == labels).sum().item()

    average_val_loss = val_loss / len(val_dataloader)
    accuracy = correct_preds / len(val_dataset)
    print(f"Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}")


<ipython-input-83-ef9e455f6b36>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Batch 1/219, Label Counts: {0: 12, 1: 2, 2: 2}
Epoch 1/3, Batch 1/219, Loss: 1.2964, Time: 1.56 seconds
Batch 2/219, Label Counts: {0: 13, 1: 2, 2: 1}
Epoch 1/3, Batch 2/219, Loss: 0.9985, Time: 1.53 seconds
Batch 3/219, Label Counts: {0: 12, 1: 4}
Epoch 1/3, Batch 3/219, Loss: 0.9229, Time: 1.53 seconds
Batch 4/219, Label Counts: {0: 12, 1: 3, 2: 1}
Epoch 1/3, Batch 4/219, Loss: 0.7902, Time: 1.55 seconds
Batch 5/219, Label Counts: {0: 11, 1: 1, 2: 4}
Epoch 1/3, Batch 5/219, Loss: 0.9526, Time: 1.57 seconds
Batch 6/219, Label Counts: {0: 11, 1: 3, 2: 2}
Epoch 1/3, Batch 6/219, Loss: 0.8320, Time: 1.58 seconds
Batch 7/219, Label Counts: {0: 12, 1: 3, 2: 1}
Epoch 1/3, Batch 7/219, Loss: 0.6708, Time: 1.58 seconds
Batch 8/219, Label Counts: {0: 13, 1: 2, 2: 1}
Epoch 1/3, Batch 8/219, Loss: 0.5837, Time: 1.57 seconds
Batch 9/219, Label Counts: {0: 10, 1: 5, 2: 1}
Epoch 1/3, Batch 9/219, Loss: 0.9917, Time: 1.61 seconds
Batch 10/219, Label Counts: {0: 12, 1: 2, 2: 2}
Epoch 1/3, Batch 10/21

In [ ]:
model.save_pretrained("./trained_models/new_train_unblanced_chunked.csv")

### **Evaluation**

In [ ]:
eval = pd.read_csv('evaluation.csv')
report_scores = pd.read_csv('evaluation.csv')

del eval['Unnamed: 0']

del report_scores['Unnamed: 0']
eval

,LogContent,RootCause
0,metrics2.impl.MetricsConfig: loaded propert...,DiskFull
1,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection
2,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection
3,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
4,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
...,...,...
147,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
148,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
149,metrics2.impl.MetricsConfig: loaded propert...,MachineDown
150,metrics2.impl.MetricsConfig: loaded propert...,MachineDown


In [ ]:
# Define the mapping for label names to numerical IDs
label_mapping = {
    'MachineDown': 0,
    'NetworkDisconnection': 1,
    'DiskFull': 2
}

# Set the 'labels' column based on the mapping
report_scores['labels'] = report_scores['RootCause'].map(lambda x: label_mapping[x.strip()])

# Create dictionaries for label mappings
id2label = {id: label for label, id in label_mapping.items()}
label2id = {label: id for label, id in label_mapping.items()}
report_scores

,LogContent,RootCause,labels
0,metrics2.impl.MetricsConfig: loaded propert...,DiskFull,2
1,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1
2,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1
3,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
4,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
...,...,...,...
147,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
148,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
149,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0
150,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0


### **Chunking the data**

In [ ]:
# Assuming you have a DataFrame named eval
# eval['Related_ID'] = range(1, len(eval) + 1)
report_scores['Related_ID'] = range(1, len(report_scores) + 1)

report_scores

,LogContent,RootCause,labels,Related_ID
0,metrics2.impl.MetricsConfig: loaded propert...,DiskFull,2,1
1,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1,2
2,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1,3
3,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,4
4,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,5
...,...,...,...,...
147,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,148
148,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,149
149,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,150
150,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,151


### **Chunking with Memory**

In [ ]:
# Function to chunk text with overlap and assign the Sample_ID
def chunk_text_with_overlap_and_id(row, tokenizer, chunk_size=508, overlap_percentage=30):
    # Tokenize the input text
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(row['LogContent'])))

    # Exclude [CLS] and [SEP] tokens
    tokens = [token for token in tokens if token not in ['[CLS]', '[SEP]']]

    # Calculate the overlap in tokens
    overlap_tokens = int(chunk_size * overlap_percentage / 100)

    # Chunk the tokens with overlap
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size - overlap_tokens)]

    # Convert token chunks back to text chunks
    text_chunks = [tokenizer.decode(tokenizer.convert_tokens_to_ids(chunk)) for chunk in chunks]

    # Assign Sample_ID to each chunk
    chunk_ids = [row['Related_ID']] * len(chunks)

    # Create a DataFrame for the current row
    df = pd.DataFrame({'LogContent': text_chunks, 'RootCause': [row['RootCause']] * len(text_chunks), 'labels': [row['labels']] * len(text_chunks), 'Related_ID': chunk_ids})

    return df

# Apply the chunking function to each row and concatenate the results
chunked_logs = pd.concat([chunk_text_with_overlap_and_id(row, tokenizer) for _, row in report_scores.iterrows()], ignore_index=True)

# Reorder the columns
chunked_logs = chunked_logs[['LogContent', 'RootCause', 'labels', 'Related_ID']]


Token indices sequence length is longer than the specified maximum sequence length for this model (1759 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
chunked_logs

,LogContent,RootCause,labels,Related_ID
0,metrics2. impl. metricsconfig : loaded propert...,DiskFull,2,1
1,maptask : ( equator ) 0 kvi 26214396 ( 1048575...,DiskFull,2,1
2,= 11165556 ( 44662224 ) ; kvend = 24952068 ( 9...,DiskFull,2,1
3,##80708 ( 29122832 ) ; kvend = 21067396 ( 8426...,DiskFull,2,1
4,##6 ( 13581584 ) ; kvend = 17182048 ( 68728192...,DiskFull,2,1
...,...,...,...,...
952,##3 _ 0003 _ m _ 000002 _ 0 decomp : 216991624...,MachineDown,0,152
953,: assigned 1 of 1 to 04dn8iq. fareast. corp. m...,MachineDown,0,152
954,##lerimpl : assigned 1 of 1 to minint - fnanli...,MachineDown,0,152
955,. ondiskmapoutput : read 216999713 bytes from ...,MachineDown,0,152


### **Normal Chunking**

In [ ]:
def chunk_text_with_overlap(text, tokenizer, chunk_size=512, overlap_percentage=30):
    # Tokenize the input text
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))

    # Exclude [CLS] and [SEP] tokens
    tokens = [token for token in tokens if token not in ['[CLS]', '[SEP]']]

    # Calculate the overlap in tokens
    overlap_tokens = int(chunk_size * overlap_percentage / 100)

    # Chunk the tokens with overlap
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size - overlap_tokens)]

    # Convert token chunks back to text chunks
    text_chunks = [tokenizer.decode(tokenizer.convert_tokens_to_ids(chunk)) for chunk in chunks]

    return text_chunks

In [ ]:
eval['TextChunks'] = eval['LogContent'].apply(lambda x: chunk_text_with_overlap(x, tokenizer, chunk_size=508, overlap_percentage=30))

In [ ]:
chunked_logs_no_id = pd.DataFrame(columns=['LogContent', 'RootCause'])
tmp = []
for index, row in eval.iterrows():
    root_cause = row['RootCause']
    log_chunks = row['TextChunks']

    # Create a DataFrame for the current row
    df = pd.DataFrame({'LogContent': log_chunks, 'RootCause': [root_cause] * len(log_chunks)})

    # Append to the list
    tmp.append(df)

# Concatenate the list of DataFrames into a single DataFrame
chunked_logs_no_id = pd.concat(tmp, ignore_index=True)

# Reorder the columns
chunked_logs_no_id = chunked_logs_no_id[['LogContent', 'RootCause']]

In [ ]:
# Apply tokenizer.convert_ids_to_tokens and calculate the length of tokens
chunked_logs['token_numbers'] = chunked_logs['LogContent'].apply(lambda x: len(tokenizer.convert_ids_to_tokens(tokenizer.encode(x))))

In [ ]:
chunked_logs[chunked_logs['token_numbers']<50].count()

LogContent       8
RootCause        8
token_numbers    8
dtype: int64

In [ ]:
del chunked_logs['token_numbers']

In [ ]:
eval = chunked_logs_no_id
eval

,LogContent,RootCause
0,metrics2. impl. metricsconfig : loaded propert...,DiskFull
1,maptask : ( equator ) 0 kvi 26214396 ( 1048575...,DiskFull
2,= 11165556 ( 44662224 ) ; kvend = 24952068 ( 9...,DiskFull
3,##80708 ( 29122832 ) ; kvend = 21067396 ( 8426...,DiskFull
4,##6 ( 13581584 ) ; kvend = 17182048 ( 68728192...,DiskFull
...,...,...
952,##3 _ 0003 _ m _ 000002 _ 0 decomp : 216991624...,MachineDown
953,: assigned 1 of 1 to 04dn8iq. fareast. corp. m...,MachineDown
954,##lerimpl : assigned 1 of 1 to minint - fnanli...,MachineDown
955,. ondiskmapoutput : read 216999713 bytes from ...,MachineDown


In [ ]:
# Define the mapping for label names to numerical IDs
label_mapping = {
    'MachineDown': 0,
    'NetworkDisconnection': 1,
    'DiskFull': 2
}

# Set the 'labels' column based on the mapping
eval['labels'] = eval['RootCause'].map(lambda x: label_mapping[x.strip()])

# Create dictionaries for label mappings
id2label = {id: label for label, id in label_mapping.items()}
label2id = {label: id for label, id in label_mapping.items()}
eval

,LogContent,RootCause,labels
0,metrics2. impl. metricsconfig : loaded propert...,DiskFull,2
1,maptask : ( equator ) 0 kvi 26214396 ( 1048575...,DiskFull,2
2,= 11165556 ( 44662224 ) ; kvend = 24952068 ( 9...,DiskFull,2
3,##80708 ( 29122832 ) ; kvend = 21067396 ( 8426...,DiskFull,2
4,##6 ( 13581584 ) ; kvend = 17182048 ( 68728192...,DiskFull,2
...,...,...,...
952,##3 _ 0003 _ m _ 000002 _ 0 decomp : 216991624...,MachineDown,0
953,: assigned 1 of 1 to 04dn8iq. fareast. corp. m...,MachineDown,0
954,##lerimpl : assigned 1 of 1 to minint - fnanli...,MachineDown,0
955,. ondiskmapoutput : read 216999713 bytes from ...,MachineDown,0


In [ ]:
class CustomDatasetWithID(Dataset):
    def __init__(self, texts, labels, related_ids, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
        self.labels = labels
        self.related_ids = related_ids

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        item['related_ids'] = torch.tensor(self.related_ids[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
report_scores

,LogContent,RootCause,labels,Related_ID
0,metrics2.impl.MetricsConfig: loaded propert...,DiskFull,2,1
1,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1,2
2,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1,3
3,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,4
4,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,5
...,...,...,...,...
147,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,148
148,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,149
149,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,150
150,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,151


In [ ]:
with_ID_dataset = CustomDatasetWithID(texts=list(chunked_logs['LogContent']), labels=chunked_logs['labels'], related_ids=chunked_logs['Related_ID'], tokenizer=tokenizer)
evaluation_dataset = CustomDataset(texts=list(eval['LogContent']), labels=eval['labels'], tokenizer=tokenizer)

# Create DataLoader for eval dataset
with_ID_dataloader = DataLoader(with_ID_dataset, batch_size=16, shuffle=False)
eval_dataloader = DataLoader(evaluation_dataset, batch_size=16, shuffle=False)

In [ ]:
# Load the trained model
model_path = "./trained_models/trained_on_chunks"
loaded_model = BertForSequenceClassification.from_pretrained(model_path)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# loaded_model.eval()
model.eval()

all_preds = []
all_labels = []
all_confidences = []

# Initialize an empty list to store batch-wise results
batch_results = []

with torch.no_grad():
    for batch in with_ID_dataloader:
        input_ids, attention_mask, labels, related_ids = (
            batch['input_ids'], batch['attention_mask'], batch['labels'], batch['related_ids']
        )
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # outputs = loaded_model(input_ids, attention_mask=attention_mask)
        outputs = model(input_ids, attention_mask=attention_mask)

        preds = torch.argmax(outputs.logits, dim=1)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)

        # Extend lists with batch-wise results
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_confidences.extend(probabilities.max(dim=1).values.cpu().numpy())

        # Store batch-wise results in a DataFrame
        batch_results.append(pd.DataFrame({'Related_ID': related_ids, 'Predicted_Label': preds.cpu().numpy(),
                                           'Confidence': probabilities.max(dim=1).values.cpu().numpy(),
                                           'True_Label': labels.cpu().numpy()}))

# Concatenate batch-wise DataFrames into the final DataFrame
result_df = pd.concat(batch_results, ignore_index=True)

# Generate a single label for each original data based on the predicted labels and confidence levels
final_predictions = []
for related_id, group in result_df.groupby('Related_ID'):
    # Choose the label with the highest confidence
    # max_confidence_idx = group['Confidence'].idxmax()
    # max_confidence_idx = group['Confidence'].idxmin()
    # final_predictions.append({'Related_ID': related_id, 'Final_Predicted_Label': result_df.loc[max_confidence_idx, 'Predicted_Label']})

    # Apply the rule to calculate the score for each label
    label_stats = group.groupby('Predicted_Label').agg(cnt=('Predicted_Label', 'size'), conf_mean=('Confidence', 'mean'))
    label_stats['score'] = label_stats['cnt']**2 * label_stats['conf_mean']

    # Find the label with the maximum score
    max_score_label = label_stats['score'].idxmax()

    final_predictions.append({'Related_ID': related_id, 'Final_Predicted_Label': max_score_label})

# Convert the final predictions to a DataFrame
final_predictions_df = pd.DataFrame(final_predictions)

# Optionally, you can merge the final predictions with the original eval dataset
# final_eval_dataset = pd.merge(eval, final_predictions_df, on='Related_ID', how='left')

final_predictions_df

<ipython-input-98-04c5fda46df1>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-98-04c5fda46df1>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-98-04c5fda46df1>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-98-04c5fda46df1>:8: UserWarning: To copy construct from a 

,Related_ID,Final_Predicted_Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
147,148,0
148,149,0
149,150,0
150,151,0


In [ ]:
result_df

,Related_ID,Predicted_Label,Confidence,True_Label
0,1,2,0.982983,2
1,1,0,0.645117,2
2,1,0,0.636173,2
3,1,0,0.640284,2
4,1,2,0.904477,2
...,...,...,...,...
952,152,0,0.961143,0
953,152,0,0.973201,0
954,152,0,0.986395,0
955,152,0,0.959202,0


In [ ]:
final_predictions_df

,Related_ID,Final_Predicted_Label
0,1,0
1,2,0
2,3,1
3,4,0
4,5,0
...,...,...
147,148,0
148,149,0
149,150,0
150,151,0


In [ ]:
final_eval_dataset = pd.merge(report_scores, final_predictions_df, on='Related_ID', how='left')

In [ ]:
report_scores

,LogContent,RootCause,labels,Related_ID
0,metrics2.impl.MetricsConfig: loaded propert...,DiskFull,2,1
1,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1,2
2,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1,3
3,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,4
4,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,5
...,...,...,...,...
147,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,148
148,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,149
149,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,150
150,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,151


In [ ]:
final_eval_dataset

,LogContent,RootCause,labels,Related_ID,Final_Predicted_Label
0,metrics2.impl.MetricsConfig: loaded propert...,DiskFull,2,1,0
1,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1,2,0
2,metrics2.impl.MetricsConfig: loaded propert...,NetworkDisconnection,1,3,1
3,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,4,0
4,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,5,0
...,...,...,...,...,...
147,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,148,0
148,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,149,0
149,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,150,0
150,metrics2.impl.MetricsConfig: loaded propert...,MachineDown,0,151,0


In [ ]:
print(classification_report(final_eval_dataset['labels'], final_eval_dataset['Final_Predicted_Label']))

              precision    recall  f1-score   support

           0       0.62      1.00      0.76        94
           1       0.00      0.00      0.00        25
           2       0.00      0.00      0.00        33

    accuracy                           0.62       152
   macro avg       0.21      0.33      0.25       152
weighted avg       0.38      0.62      0.47       152



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# loaded_model.eval()
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_dataloader:
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # outputs = loaded_model(input_ids, attention_mask=attention_mask)
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds))

<ipython-input-99-db7b66662265>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


              precision    recall  f1-score   support

           0       0.57      1.00      0.73       548
           1       0.00      0.00      0.00       210
           2       0.00      0.00      0.00       199

    accuracy                           0.57       957
   macro avg       0.19      0.33      0.24       957
weighted avg       0.33      0.57      0.42       957



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.save_pretrained("./trained_models/trained_on_chunks")

# Cross Validation

In [ ]:
chunked_logs = pd.read_csv('chunked_new.csv')
chunked_logs
del chunked_logs['Unnamed: 0']

In [ ]:
# Define the mapping for label names to numerical IDs
label_mapping = {
    'MachineDown': 0,
    'NetworkDisconnection': 1,
    'DiskFull': 2
}
NUM_LABELS = 3
# Set the 'labels' column based on the mapping
chunked_logs['labels'] = chunked_logs['RootCause'].map(lambda x: label_mapping[x.strip()])

# Create dictionaries for label mappings
id2label = {id: label for label, id in label_mapping.items()}
label2id = {label: id for label, id in label_mapping.items()}
chunked_logs

,LogContent,RootCause,labels
0,metrics2. impl. metricsconfig : loaded propert...,MachineDown,0
1,. maptask : ( equator ) 0 kvi 26214396 ( 10485...,MachineDown,0
2,##k : kvstart = 11165712 ( 44662848 ) ; kvend ...,MachineDown,0
3,: kvstart = 7281300 ( 29125200 ) ; kvend = 210...,MachineDown,0
4,##388 ( 13585552 ) ; kvend = 17183192 ( 687327...,MachineDown,0
...,...,...,...
4369,maptask : ( equator ) 0 kvi 26214396 ( 1048575...,MachineDown,0
4370,kvstart = 14330776 ( 57323104 ) ; kvend = 5417...,MachineDown,0
4371,##t = 16762552 ( 67050208 ) ; kvend = 7853256 ...,MachineDown,0
4372,##4 ( 76802656 ) ; kvend = 10290256 ( 41161024...,MachineDown,0


In [ ]:
# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=NUM_LABELS
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Number of folds
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=50)

In [ ]:
# Lists to store metrics for each fold without balancing
all_train_losses = []
all_val_losses = []
all_accuracies = []
num_epochs = 3
# K-fold cross-validation loop
for fold, (train_idx, val_idx) in enumerate(kf.split(chunked_logs)):
    print(f"Fold {fold + 1}/{num_folds}")

     # Reinitialize the model for each fold
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=NUM_LABELS
    )
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    # Split the data into train and validation datasets for the current fold
    train_fold, val_fold = chunked_logs.iloc[train_idx].reset_index(drop=True), chunked_logs.iloc[val_idx].reset_index(drop=True)

    # Create CustomDataset instances for training and validation
    train_dataset = CustomDataset(texts=list(train_fold['LogContent']), labels=train_fold['labels'], tokenizer=tokenizer)
    val_dataset = CustomDataset(texts=list(val_fold['LogContent']), labels=val_fold['labels'], tokenizer=tokenizer)

    # Create DataLoader
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)

    # Training loop for the current fold
    train_losses_fold = []
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        start_time = time.time()

        for batch_idx, batch in enumerate(train_dataloader):
            batch_start_time = time.time()

            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Print label counts for the current batch
            # label_counts = {label: count for label, count in zip(*np.unique(labels.cpu().numpy(), return_counts=True))}
            # print(f"Batch {batch_idx + 1}/{len(train_dataloader)}, Label Counts: {label_counts}")

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            batch_end_time = time.time()
            batch_time = batch_end_time - batch_start_time

            # Print training progress for each batch
            print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")
        average_loss = total_loss / len(train_dataloader)
        epoch_end_time = time.time()
        epoch_time = epoch_end_time - start_time

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")

        train_losses_fold.append(average_loss)

        # Validation
        model.eval()
        val_loss = 0.0
        correct_preds = 0

        with torch.no_grad():
            for batch_idx, batch in enumerate(val_dataloader):
                input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

                # Forward pass
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                val_loss += outputs.loss.item()

                # Calculate accuracy
                preds = torch.argmax(outputs.logits, dim=1)
                correct_preds += (preds == labels).sum().item()
        average_val_loss = val_loss / len(val_dataloader)
        accuracy = correct_preds / len(val_fold)
        print(f"Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}")

        all_val_losses.append(average_val_loss)
        all_accuracies.append(accuracy)

    all_train_losses.append(train_losses_fold)

Fold 1/5


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-99-db7b66662265>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/3, Batch 1/219, Loss: 1.4525, Time: 1.55 seconds
Epoch 1/3, Batch 2/219, Loss: 0.8376, Time: 1.52 seconds
Epoch 1/3, Batch 3/219, Loss: 0.8402, Time: 1.54 seconds
Epoch 1/3, Batch 4/219, Loss: 0.7677, Time: 1.55 seconds
Epoch 1/3, Batch 5/219, Loss: 0.9576, Time: 1.56 seconds
Epoch 1/3, Batch 6/219, Loss: 0.9443, Time: 1.58 seconds
Epoch 1/3, Batch 7/219, Loss: 0.8229, Time: 1.58 seconds
Epoch 1/3, Batch 8/219, Loss: 0.8353, Time: 1.60 seconds
Epoch 1/3, Batch 9/219, Loss: 0.8526, Time: 1.59 seconds
Epoch 1/3, Batch 10/219, Loss: 0.7980, Time: 1.60 seconds
Epoch 1/3, Batch 11/219, Loss: 0.6415, Time: 1.61 seconds
Epoch 1/3, Batch 12/219, Loss: 0.7352, Time: 1.60 seconds
Epoch 1/3, Batch 13/219, Loss: 1.3825, Time: 1.62 seconds
Epoch 1/3, Batch 14/219, Loss: 0.7935, Time: 1.62 seconds
Epoch 1/3, Batch 15/219, Loss: 1.0958, Time: 1.61 seconds
Epoch 1/3, Batch 16/219, Loss: 0.8796, Time: 1.64 seconds
Epoch 1/3, Batch 17/219, Loss: 0.8249, Time: 1.67 seconds
Epoch 1/3, Batch 18/219

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=512)
class CustomDatasetForSMOTE(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
        # self.labels = labels
        self.labels = torch.tensor(labels).clone().detach()

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Lists to store metrics for each fold with balancing (using SMOTE)
all_train_losses = []
all_val_losses = []
all_accuracies = []
num_epochs = 3
# K-fold cross-validation loop
for fold, (train_idx, val_idx) in enumerate(kf.split(chunked_logs)):
    print(f"Fold {fold + 1}/{num_folds}")

     # Reinitialize the model for each fold
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=NUM_LABELS
    )
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    # Split the data into train and validation datasets for the current fold
    train_fold, val_fold = chunked_logs.iloc[train_idx].reset_index(drop=True), chunked_logs.iloc[val_idx].reset_index(drop=True)

    # Create CustomDataset instances for training and validation
    train_dataset = CustomDatasetForSMOTE(texts=list(train_fold['LogContent']), labels=train_fold['labels'], tokenizer=tokenizer)
    val_dataset = CustomDatasetForSMOTE(texts=list(val_fold['LogContent']), labels=val_fold['labels'], tokenizer=tokenizer)

    # Step 1: Extract features and labels
    features = torch.cat([train_dataset[i]['input_ids'].reshape(1, -1) for i in range(len(train_dataset))], dim=0)
    labels = torch.tensor(train_dataset[:]['labels'])

    # Step 2: Apply SMOTE
    smote = SMOTE(random_state=40)
    features_resampled, labels_resampled = smote.fit_resample(features, labels)

    # Step 3: Convert feature vectors back to text
    text_data_resampled = [tokenizer.decode(input_ids) for input_ids in features_resampled]

    # Step 4: Create a new dataset
    train_dataset_resampled = CustomDatasetForSMOTE(text_data_resampled, labels_resampled, tokenizer)

    # Create DataLoader
    train_dataloader = DataLoader(train_dataset_resampled, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)

    # Training loop for the current fold
    train_losses_fold = []
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        start_time = time.time()

        for batch_idx, batch in enumerate(train_dataloader):
            batch_start_time = time.time()

            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Print label counts for the current batch
            # label_counts = {label: count for label, count in zip(*np.unique(labels.cpu().numpy(), return_counts=True))}
            # print(f"Batch {batch_idx + 1}/{len(train_dataloader)}, Label Counts: {label_counts}")

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            batch_end_time = time.time()
            batch_time = batch_end_time - batch_start_time

            # Print training progress for each batch
            print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")
        average_loss = total_loss / len(train_dataloader)
        epoch_end_time = time.time()
        epoch_time = epoch_end_time - start_time

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")

        train_losses_fold.append(average_loss)

        # Validation
        model.eval()
        val_loss = 0.0
        correct_preds = 0

        with torch.no_grad():
            for batch_idx, batch in enumerate(val_dataloader):
                input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

                # Forward pass
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                val_loss += outputs.loss.item()

                # Calculate accuracy
                preds = torch.argmax(outputs.logits, dim=1)
                correct_preds += (preds == labels).sum().item()
        average_val_loss = val_loss / len(val_dataloader)
        accuracy = correct_preds / len(val_fold)
        print(f"Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}")

        all_val_losses.append(average_val_loss)
        all_accuracies.append(accuracy)

    all_train_losses.append(train_losses_fold)

Fold 1/5


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-13-3adbf41d52d9>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-14-ba2ef8751743>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(train_dataset[:]['labels'])
<ipython-input-13-3adbf41d52d9>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or s

Epoch 1/3, Batch 1/448, Loss: 1.2360, Time: 1.66 seconds
Epoch 1/3, Batch 2/448, Loss: 1.0984, Time: 1.64 seconds
Epoch 1/3, Batch 3/448, Loss: 1.1273, Time: 1.64 seconds
Epoch 1/3, Batch 4/448, Loss: 1.1906, Time: 1.67 seconds
Epoch 1/3, Batch 5/448, Loss: 1.0822, Time: 1.67 seconds
Epoch 1/3, Batch 6/448, Loss: 1.0543, Time: 1.68 seconds
Epoch 1/3, Batch 7/448, Loss: 1.0055, Time: 1.69 seconds
Epoch 1/3, Batch 8/448, Loss: 1.0282, Time: 1.69 seconds
Epoch 1/3, Batch 9/448, Loss: 1.1811, Time: 1.72 seconds
Epoch 1/3, Batch 10/448, Loss: 0.9440, Time: 1.73 seconds
Epoch 1/3, Batch 11/448, Loss: 0.9929, Time: 1.73 seconds
Epoch 1/3, Batch 12/448, Loss: 1.0767, Time: 1.74 seconds
Epoch 1/3, Batch 13/448, Loss: 0.9116, Time: 1.74 seconds
Epoch 1/3, Batch 14/448, Loss: 0.9396, Time: 1.76 seconds
Epoch 1/3, Batch 15/448, Loss: 0.9081, Time: 1.76 seconds
Epoch 1/3, Batch 16/448, Loss: 0.9754, Time: 1.78 seconds
Epoch 1/3, Batch 17/448, Loss: 0.9880, Time: 1.79 seconds
Epoch 1/3, Batch 18/448

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-14-ba2ef8751743>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(train_dataset[:]['labels'])
<ipython-input-13-3adbf41d52d9>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch 1/3, Batch 1/447, Loss: 1.2547, Time: 1.65 seconds
Epoch 1/3, Batch 2/447, Loss: 1.2984, Time: 1.66 seconds
Epoch 1/3, Batch 3/447, Loss: 0.9978, Time: 1.66 seconds
Epoch 1/3, Batch 4/447, Loss: 1.1840, Time: 1.68 seconds
Epoch 1/3, Batch 5/447, Loss: 1.0817, Time: 1.69 seconds
Epoch 1/3, Batch 6/447, Loss: 1.0352, Time: 1.69 seconds
Epoch 1/3, Batch 7/447, Loss: 0.9546, Time: 1.71 seconds
Epoch 1/3, Batch 8/447, Loss: 0.9682, Time: 1.72 seconds
Epoch 1/3, Batch 9/447, Loss: 0.9378, Time: 1.74 seconds
Epoch 1/3, Batch 10/447, Loss: 1.0785, Time: 1.75 seconds
Epoch 1/3, Batch 11/447, Loss: 1.0574, Time: 1.76 seconds
Epoch 1/3, Batch 12/447, Loss: 1.0554, Time: 1.77 seconds
Epoch 1/3, Batch 13/447, Loss: 1.0053, Time: 1.78 seconds
Epoch 1/3, Batch 14/447, Loss: 0.8946, Time: 1.79 seconds
Epoch 1/3, Batch 15/447, Loss: 0.9067, Time: 1.81 seconds
Epoch 1/3, Batch 16/447, Loss: 0.8215, Time: 1.80 seconds
Epoch 1/3, Batch 17/447, Loss: 1.1692, Time: 1.82 seconds
Epoch 1/3, Batch 18/447

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-14-ba2ef8751743>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(train_dataset[:]['labels'])
<ipython-input-13-3adbf41d52d9>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch 1/3, Batch 1/448, Loss: 1.1331, Time: 1.65 seconds
Epoch 1/3, Batch 2/448, Loss: 1.1015, Time: 1.63 seconds
Epoch 1/3, Batch 3/448, Loss: 0.9966, Time: 1.64 seconds
Epoch 1/3, Batch 4/448, Loss: 0.9173, Time: 1.66 seconds
Epoch 1/3, Batch 5/448, Loss: 1.1090, Time: 1.67 seconds
Epoch 1/3, Batch 6/448, Loss: 1.2376, Time: 1.68 seconds
Epoch 1/3, Batch 7/448, Loss: 1.0440, Time: 1.68 seconds
Epoch 1/3, Batch 8/448, Loss: 1.0115, Time: 1.70 seconds
Epoch 1/3, Batch 9/448, Loss: 0.8268, Time: 1.71 seconds
Epoch 1/3, Batch 10/448, Loss: 0.9046, Time: 1.72 seconds
Epoch 1/3, Batch 11/448, Loss: 0.7123, Time: 1.74 seconds
Epoch 1/3, Batch 12/448, Loss: 0.8827, Time: 1.75 seconds
Epoch 1/3, Batch 13/448, Loss: 0.9941, Time: 1.75 seconds
Epoch 1/3, Batch 14/448, Loss: 0.8919, Time: 1.77 seconds
Epoch 1/3, Batch 15/448, Loss: 0.8519, Time: 1.77 seconds
Epoch 1/3, Batch 16/448, Loss: 0.8926, Time: 1.78 seconds
Epoch 1/3, Batch 17/448, Loss: 0.8073, Time: 1.79 seconds
Epoch 1/3, Batch 18/448

In [ ]:
import matplotlib.pyplot as plt

# Calculate mean accuracy and mean validation loss across all folds
mean_accuracy_across_folds = np.mean(all_accuracies)
mean_val_loss_across_folds = np.mean(all_val_losses)

print(f"Mean Accuracy Across Folds: {mean_accuracy_across_folds:.4f}")
print(f"Mean Validation Loss Across Folds: {mean_val_loss_across_folds:.4f}")

# Plot training and validation losses for each fold
plt.figure(figsize=(15, 6))

# Plot training losses
plt.subplot(1, 2, 1)
for fold, train_loss_fold in enumerate(all_train_losses, start=1):
    plt.plot(range(1, num_epochs + 1), train_loss_fold, label=f"Fold {fold} - Train Loss")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Across Folds")
plt.legend()

# Plot validation losses for each fold and epoch
plt.figure(figsize=(12, 6))
for fold in range(num_folds):
    fold_val_losses = all_val_losses[fold * num_epochs: (fold + 1) * num_epochs]
    plt.plot(range(1, len(fold_val_losses) + 1), fold_val_losses, marker='o', label=f"Fold {fold + 1}")

plt.xlabel("Epoch")
plt.ylabel("Validation Loss")
plt.title("Validation Loss Across Folds and Epochs")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
num_epochs = 3

# Number of folds
num_folds = 5

# Plot each fold
plt.figure(figsize=(12, 8))
for fold in range(num_folds):
    start_idx = fold * num_epochs
    end_idx = (fold + 1) * num_epochs
    accuracies_fold = all_accuracies[start_idx:end_idx]
    plt.plot(range(1, num_epochs + 1), accuracies_fold, label=f"Fold {fold + 1}")

plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy for Each Fold Across Epochs")
plt.legend()
plt.show()


## SMOTE

In [ ]:
chunked_logs = pd.read_csv('chunked_new.csv')
chunked_logs

,Unnamed: 0,LogContent,RootCause
0,0,metrics2. impl. metricsconfig : loaded propert...,MachineDown
1,1,. maptask : ( equator ) 0 kvi 26214396 ( 10485...,MachineDown
2,2,##k : kvstart = 11165712 ( 44662848 ) ; kvend ...,MachineDown
3,3,: kvstart = 7281300 ( 29125200 ) ; kvend = 210...,MachineDown
4,4,##388 ( 13585552 ) ; kvend = 17183192 ( 687327...,MachineDown
...,...,...,...
4369,4369,maptask : ( equator ) 0 kvi 26214396 ( 1048575...,MachineDown
4370,4370,kvstart = 14330776 ( 57323104 ) ; kvend = 5417...,MachineDown
4371,4371,##t = 16762552 ( 67050208 ) ; kvend = 7853256 ...,MachineDown
4372,4372,##4 ( 76802656 ) ; kvend = 10290256 ( 41161024...,MachineDown


In [ ]:
# Define the mapping for label names to numerical IDs
label_mapping = {
    'MachineDown': 0,
    'NetworkDisconnection': 1,
    'DiskFull': 2
}

# Set the 'labels' column based on the mapping
chunked_logs['labels'] = chunked_logs['RootCause'].map(lambda x: label_mapping[x.strip()])

# Create dictionaries for label mappings
id2label = {id: label for label, id in label_mapping.items()}
label2id = {label: id for label, id in label_mapping.items()}
chunked_logs

,Unnamed: 0,LogContent,RootCause,labels
0,0,metrics2. impl. metricsconfig : loaded propert...,MachineDown,0
1,1,. maptask : ( equator ) 0 kvi 26214396 ( 10485...,MachineDown,0
2,2,##k : kvstart = 11165712 ( 44662848 ) ; kvend ...,MachineDown,0
3,3,: kvstart = 7281300 ( 29125200 ) ; kvend = 210...,MachineDown,0
4,4,##388 ( 13585552 ) ; kvend = 17183192 ( 687327...,MachineDown,0
...,...,...,...,...
4369,4369,maptask : ( equator ) 0 kvi 26214396 ( 1048575...,MachineDown,0
4370,4370,kvstart = 14330776 ( 57323104 ) ; kvend = 5417...,MachineDown,0
4371,4371,##t = 16762552 ( 67050208 ) ; kvend = 7853256 ...,MachineDown,0
4372,4372,##4 ( 76802656 ) ; kvend = 10290256 ( 41161024...,MachineDown,0


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=512)
class CustomDatasetForSMOTE(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
        # self.labels = labels
        self.labels = torch.tensor(labels).clone().detach()

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Split the data into train and validation datasets (80-20 split)
train, val = train_test_split(chunked_logs, test_size=0.2, random_state=45, shuffle=True)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

In [ ]:
train['labels'].value_counts()

0    2387
1     589
2     523
Name: labels, dtype: int64

In [ ]:
# Create CustomDataset instances for training and validation
train_dataset = CustomDatasetForSMOTE(texts=list(train['LogContent']), labels=train['labels'], tokenizer=tokenizer)
val_dataset = CustomDatasetForSMOTE(texts=list(val['LogContent']), labels=val['labels'], tokenizer=tokenizer)

In [ ]:
# Access the encodings of the first sample in the dataset
train_dataset

### applying SMOTE

In [ ]:
# Step 1: Extract features and labels
features = torch.cat([train_dataset[i]['input_ids'].reshape(1, -1) for i in range(len(train_dataset))], dim=0)
labels = torch.tensor(train_dataset[:]['labels'])

<ipython-input-6-3adbf41d52d9>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-14-f108629a96d8>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(train_dataset[:]['labels'])


In [ ]:
features

tensor([[  101, 26640,  1007,  ...,   102,     0,     0],
        [  101,  1001,  1001,  ...,  1033,  4949,   102],
        [  101,  1001,  1001,  ...,     0,     0,     0],
        ...,
        [  101,  3535,  1035,  ...,   102,     0,     0],
        [  101, 12046,  2015,  ...,   102,     0,     0],
        [  101,  1001,  1001,  ...,     0,     0,     0]])

In [ ]:
# Step 2: Apply SMOTE
smote = SMOTE(random_state=40)
features_resampled, labels_resampled = smote.fit_resample(features, labels)

In [ ]:
# Step 3: Convert feature vectors back to text
text_data_resampled = [tokenizer.decode(input_ids) for input_ids in features_resampled]

In [ ]:
# Assuming labels_resampled is your NumPy array
unique_values, counts = np.unique(labels_resampled, return_counts=True)

# Print unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"Label {value}: Count = {count}")

Label 0: Count = 2387
Label 1: Count = 2387
Label 2: Count = 2387


In [ ]:
len(labels_resampled)

7161

In [ ]:
# Step 4: Create a new dataset
train_dataset_resampled = CustomDatasetForSMOTE(text_data_resampled, labels_resampled, tokenizer)

In [ ]:
# Create DataLoader
train_dataloader = DataLoader(train_dataset_resampled, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [ ]:
train_dataset_resampled

In [ ]:
NUM_LABELS = 3
# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=NUM_LABELS
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import time
# Training loop without early stopping
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    start_time = time.time()

    # Iterate over batches in the training DataLoader
    for batch_idx, batch in enumerate(train_dataloader):
        batch_start_time = time.time()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Print label counts for the current batch
        label_counts = {label: count for label, count in zip(*np.unique(labels.cpu().numpy(), return_counts=True))}
        print(f"Batch {batch_idx + 1}/{len(train_dataloader)}, Label Counts: {label_counts}")

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time

        # Print training progress for each batch
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}, Time: {batch_time:.2f} seconds")

    average_loss = total_loss / len(train_dataloader)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - start_time

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Time: {epoch_time:.2f} seconds")

    # Validation
    model.eval()
    val_loss = 0.0
    correct_preds = 0

    # Iterate over batches in the validation DataLoader
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Calculate accuracy
            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += (preds == labels).sum().item()

    average_val_loss = val_loss / len(val_dataloader)
    accuracy = correct_preds / len(val_dataset)
    print(f"Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}")


<ipython-input-6-3adbf41d52d9>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Batch 1/448, Label Counts: {0: 8, 1: 5, 2: 3}
Epoch 1/3, Batch 1/448, Loss: 1.0042, Time: 2.65 seconds
Batch 2/448, Label Counts: {0: 8, 1: 4, 2: 4}
Epoch 1/3, Batch 2/448, Loss: 0.9106, Time: 1.35 seconds
Batch 3/448, Label Counts: {0: 5, 1: 3, 2: 8}
Epoch 1/3, Batch 3/448, Loss: 1.1417, Time: 1.35 seconds
Batch 4/448, Label Counts: {0: 6, 1: 5, 2: 5}
Epoch 1/3, Batch 4/448, Loss: 0.9712, Time: 1.36 seconds
Batch 5/448, Label Counts: {0: 4, 1: 7, 2: 5}
Epoch 1/3, Batch 5/448, Loss: 1.0177, Time: 1.35 seconds
Batch 6/448, Label Counts: {0: 6, 1: 5, 2: 5}
Epoch 1/3, Batch 6/448, Loss: 0.9191, Time: 1.35 seconds
Batch 7/448, Label Counts: {0: 3, 1: 7, 2: 6}
Epoch 1/3, Batch 7/448, Loss: 0.9517, Time: 1.35 seconds
Batch 8/448, Label Counts: {0: 8, 1: 4, 2: 4}
Epoch 1/3, Batch 8/448, Loss: 0.9210, Time: 1.36 seconds
Batch 9/448, Label Counts: {0: 6, 1: 8, 2: 2}
Epoch 1/3, Batch 9/448, Loss: 1.0019, Time: 1.36 seconds
Batch 10/448, Label Counts: {0: 8, 1: 3, 2: 5}
Epoch 1/3, Batch 10/448, L

In [ ]:
model.save_pretrained("./trained_models/SMOTE_FINAL_REPORT")